# Introduction

This code takes the statbomb data. It applies feature engineering by event type to summarise events across a match.


# Configuration

In [ ]:
# game chunk file size set to avoid code crashes
chunk_size = 250
#chunk selection
a=0
#chunk file export
b=1


In [ ]:
!pip install statsbombpy
!pip install mplsoccer
!pip install highlight_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.4/86.4 kB 4.7 MB/s eta 0:00:00


In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from statsbombpy import sb
import pandas as pd
from mplsoccer import VerticalPitch, Pitch
from highlight_text import ax_text, fig_text
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt


The statsbombpy automatically interact with repository. For more details you can access to github.


https://github.com/statsbomb/open-data

In [ ]:
df_competitions = sb.competitions()
df_competitions.head()

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,281,Germany,1. Bundesliga,male,False,False,2023/2024,2024-07-15T14:15:54.671676,2024-07-15T14:17:00.877356,2024-07-15T14:17:00.877356,2024-07-15T14:15:54.671676
1,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2024-05-19T11:11:14.192381,None,None,2024-05-19T11:11:14.192381
2,1267,107,Africa,African Cup of Nations,male,False,True,2023,2024-06-13T07:51:02.452825,None,None,2024-06-13T07:51:02.452825
3,16,4,Europe,Champions League,male,False,False,2018/2019,2024-06-12T07:44:38.559714,2021-06-13T16:17:31.694,None,2024-06-12T07:44:38.559714
4,16,1,Europe,Champions League,male,False,False,2017/2018,2024-02-13T02:35:28.134882,2021-06-13T16:17:31.694,None,2024-02-13T02:35:28.134882


Filters need setting to select matches of interest

In [ ]:
# filter games of interest
df_competitions['season_year'] = df_competitions['season_name'].str[-4:].astype(int)
df_competitions_filtered = df_competitions[
    (df_competitions['season_year'] > 2000) &
    (df_competitions['competition_international'] == False) &
    (df_competitions['competition_youth'] == False) &
    (df_competitions['competition_gender'] == 'male')
]
df_competitions_filtered

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available,season_year
0,9,281,Germany,1. Bundesliga,male,False,False,2023/2024,2024-07-15T14:15:54.671676,2024-07-15T14:17:00.877356,2024-07-15T14:17:00.877356,2024-07-15T14:15:54.671676,2024
1,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2024-05-19T11:11:14.192381,None,None,2024-05-19T11:11:14.192381,2016
3,16,4,Europe,Champions League,male,False,False,2018/2019,2024-06-12T07:44:38.559714,2021-06-13T16:17:31.694,None,2024-06-12T07:44:38.559714,2019
4,16,1,Europe,Champions League,male,False,False,2017/2018,2024-02-13T02:35:28.134882,2021-06-13T16:17:31.694,None,2024-02-13T02:35:28.134882,2018
5,16,2,Europe,Champions League,male,False,False,2016/2017,2024-02-13T02:37:32.205154,2021-06-13T16:17:31.694,None,2024-02-13T02:37:32.205154,2017
6,16,27,Europe,Champions League,male,False,False,2015/2016,2024-06-12T07:45:38.786894,2021-06-13T16:17:31.694,None,2024-06-12T07:45:38.786894,2016
7,16,26,Europe,Champions League,male,False,False,2014/2015,2024-02-12T12:49:54.914228,2021-06-13T16:17:31.694,None,2024-02-12T12:49:54.914228,2015
8,16,25,Europe,Champions League,male,False,False,2013/2014,2024-02-12T12:48:48.479157,2021-06-13T16:17:31.694,None,2024-02-12T12:48:48.479157,2014
9,16,24,Europe,Champions League,male,False,False,2012/2013,2024-02-12T12:47:34.340413,2021-06-13T16:17:31.694,None,2024-02-12T12:47:34.340413,2013
10,16,23,Europe,Champions League,male,False,False,2011/2012,2024-02-13T02:36:35.698340,2021-06-13T16:17:31.694,None,2024-02-13T02:36:35.698340,2012


https://github.com/statsbomb/open-data/blob/master/data/competitions.json





In [ ]:

import warnings

# Suppress UserWarnings related to mplsoccer
warnings.filterwarnings("ignore", category=UserWarning, module="mplsoccer")


all_matches = []
for index, row in df_competitions_filtered.iterrows():
    competition_id = row['competition_id']
    season_id = row['season_id']
    try:
        df_matches = sb.matches(competition_id=competition_id, season_id=season_id)
        all_matches.append(df_matches)
    except Exception as e:
        print(f"Could not retrieve matches for competition_id: {competition_id}, season_id: {season_id}. Error: {e}")

if all_matches:
    df_all_matches = pd.concat(all_matches, ignore_index=True)
    print(df_all_matches.head())
else:
    print("No matches data retrieved.")


df_all_matches

   match_id  match_date      kick_off              competition     season  \
0   3895302  2024-04-14  17:30:00.000  Germany - 1. Bundesliga  2023/2024   
1   3895292  2024-04-06  15:30:00.000  Germany - 1. Bundesliga  2023/2024   
2   3895333  2024-05-05  18:30:00.000  Germany - 1. Bundesliga  2023/2024   
3   3895340  2024-05-12  20:30:00.000  Germany - 1. Bundesliga  2023/2024   
4   3895348  2024-05-18  16:30:00.000  Germany - 1. Bundesliga  2023/2024   

             home_team         away_team  home_score  away_score match_status  \
0     Bayer Leverkusen     Werder Bremen           5           0    available   
1         Union Berlin  Bayer Leverkusen           0           1    available   
2  Eintracht Frankfurt  Bayer Leverkusen           1           5    available   
3               Bochum  Bayer Leverkusen           0           5    available   
4     Bayer Leverkusen          Augsburg           2           1    available   

   ...            last_updated_360 match_week comp

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3895302,2024-04-14,17:30:00.000,Germany - 1. Bundesliga,2023/2024,Bayer Leverkusen,Werder Bremen,5,0,available,...,2024-05-10T17:03:59.613154,29,Regular Season,BayArena,Harm Osmers,Xabier Alonso Olano,Ole Werner,1.1.0,2,2
1,3895292,2024-04-06,15:30:00.000,Germany - 1. Bundesliga,2023/2024,Union Berlin,Bayer Leverkusen,0,1,available,...,2024-05-12T21:08:37.897296,28,Regular Season,Stadion An der Alten Försterei,Benjamin Brand,Nenad Bjelica,Xabier Alonso Olano,1.1.0,2,2
2,3895333,2024-05-05,18:30:00.000,Germany - 1. Bundesliga,2023/2024,Eintracht Frankfurt,Bayer Leverkusen,1,5,available,...,2024-05-14T16:32:13.483516,32,Regular Season,Deutsche Bank Park,Christian Dingert,Dino Toppmöller,Xabier Alonso Olano,1.1.0,2,2
3,3895340,2024-05-12,20:30:00.000,Germany - 1. Bundesliga,2023/2024,Bochum,Bayer Leverkusen,0,5,available,...,2024-05-14T16:46:08.459843,33,Regular Season,Vonovia Ruhrstadion,Benjamin Brand,Heiko Butscher,Xabier Alonso Olano,1.1.0,2,2
4,3895348,2024-05-18,16:30:00.000,Germany - 1. Bundesliga,2023/2024,Bayer Leverkusen,Augsburg,2,1,available,...,2024-05-20T10:33:09.140760,34,Regular Season,BayArena,Matthias Jöllenbeck,Xabier Alonso Olano,Jess Christian Thorup,1.1.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2570,3878544,2015-08-23,20:45:00.000,Italy - Serie A,2015/2016,Palermo,Genoa,1,0,available,...,None,1,Regular Season,Stadio Renzo Barbera,Massimiliano Irrati,Giuseppe Iachini,Gian Piero Gasperini,1.1.0,2,2
2571,3878543,2015-08-23,20:45:00.000,Italy - Serie A,2015/2016,Inter Milan,Atalanta,1,0,available,...,None,1,Regular Season,Stadio Giuseppe Meazza,Gianpaolo Calvarese\t,Roberto Mancini,Edoardo Reja,1.1.0,2,2
2572,3878542,2015-08-23,20:45:00.000,Italy - Serie A,2015/2016,Fiorentina,AC Milan,2,0,available,...,None,1,Regular Season,Stadio Artemio Franchi \t,Paolo Valeri,Paulo Manuel Carvalho de Sousa,Siniša Mihajlović,1.1.0,2,2
2573,3878541,2015-08-22,18:00:00.000,Italy - Serie A,2015/2016,Hellas Verona,AS Roma,1,1,available,...,None,1,Regular Season,Stadio Marc''Antonio Bentegodi,Marco Guida,"Rafael Márquez Álvarez, Andrea Mandorlini",Rudi Garcia,1.1.0,2,2


In [ ]:
#take competition and match_id columns from df_all_matches

df_selected_cols = df_all_matches[['competition', 'match_id','season']]
print(df_selected_cols.head())

               competition  match_id     season
0  Germany - 1. Bundesliga   3895302  2023/2024
1  Germany - 1. Bundesliga   3895292  2023/2024
2  Germany - 1. Bundesliga   3895333  2023/2024
3  Germany - 1. Bundesliga   3895340  2023/2024
4  Germany - 1. Bundesliga   3895348  2023/2024


In [ ]:
df_wwc = df_all_matches

In [ ]:

df_wwc_subset = df_wwc[['match_id', 'kick_off', 'match_week']].copy()
df_wwc_subset['kick_off_hour'] = pd.to_datetime(df_wwc_subset['kick_off']).dt.hour
df_wwc_subset = df_wwc_subset.drop(columns=['kick_off'])
df_wwc_subset

,match_id,match_week,kick_off_hour
0,3895302,29,17.0
1,3895292,28,15.0
2,3895333,32,18.0
3,3895340,33,20.0
4,3895348,34,16.0
...,...,...,...
2570,3878544,1,20.0
2571,3878543,1,20.0
2572,3878542,1,20.0
2573,3878541,1,18.0


In [ ]:
matchids=df_wwc['match_id'].unique()
print(matchids)

[3895302 3895292 3895333 ... 3878542 3878541 3878540]


In [ ]:

# Group match ids into chunks to manage file size
match_id_chunks = [matchids[i:i + chunk_size] for i in range(0, len(matchids), chunk_size)]

# Print the names of each group (by index)
for i, chunk in enumerate(match_id_chunks):
  print(f"Group {i+1}")
  # If you want to see the match ids in each group, uncomment the line below
  print(chunk)

Group 1
[3895302 3895292 3895333 3895340 3895348 3895286 3895220 3895250 3895266
 3895275 3895180 3895134 3895121 3895107 3895074 3895139 3895086 3895320
 3895309 3895258 3895244 3895232 3895210 3895202 3895194 3895182 3895167
 3895158 3895153 3895113 3895095 3895067 3895060 3895052 3890561 3890505
 3890511 3890515 3890411 3890397 3890401 3890396 3890384 3890385 3890282
 3890287 3890269 3890268 3890402 3890564 3890563 3890562 3890560 3890559
 3890558 3890557 3890556 3890555 3890554 3890553 3890552 3890551 3890550
 3890549 3890548 3890547 3890546 3890545 3890544 3890543 3890542 3890541
 3890540 3890539 3890538 3890537 3890536 3890535 3890534 3890533 3890532
 3890531 3890530 3890529 3890528 3890527 3890526 3890525 3890524 3890523
 3890522 3890521 3890520 3890519 3890518 3890517 3890516 3890514 3890513
 3890512 3890510 3890509 3890508 3890507 3890506 3890504 3890503 3890502
 3890501 3890500 3890499 3890498 3890497 3890496 3890495 3890494 3890493
 3890492 3890491 3890490 3890489 3890488 38

In [ ]:

df_final_matches = pd.merge(df_final_matches, df_wwc[['match_id', 'home_team']], on='match_id', how='left')


In [ ]:

df_final_matches['home_team_flag'] = (df_final_matches['possession_team'] == df_final_matches['home_team']).astype(int)


In [ ]:
df_final_matches[['x','y']] = df_final_matches['location'].apply(pd.Series)
df_final_matches[['carry_end_x','carry_end_y']] = df_final_matches['carry_end_location'].apply(pd.Series)
#df_final_matches[['goalkeeper_end_x','goalkeeper_end_y']] = df_final_matches['goalkeeper_end_location'].apply(pd.Series)
df_final_matches[['pass_end_x','pass_end_y']] = df_final_matches['pass_end_location'].apply(pd.Series)

In [ ]:

df_final_matches['new_id'] = df_final_matches['match_id'].astype(str) + '_' + df_final_matches['possession'].astype(str)
df_first_event_of_possession = df_final_matches.loc[df_final_matches.groupby('new_id')['index'].idxmin()]

In [ ]:
print(df_final_matches['type'].unique())

['Starting XI' 'Half Start' 'Pass' 'Ball Receipt*' 'Carry' 'Pressure'
 'Miscontrol' 'Clearance' 'Duel' 'Foul Committed' 'Foul Won' 'Shot'
 'Goal Keeper' 'Ball Recovery' 'Block' 'Interception' 'Dribbled Past'
 'Dribble' 'Dispossessed' '50/50' 'Injury Stoppage' 'Player Off'
 'Player On' 'Half End' 'Substitution' 'Tactical Shift' 'Offside' 'Shield'
 'Error' 'Bad Behaviour' 'Referee Ball-Drop' 'Own Goal For'
 'Own Goal Against']


In [ ]:
#
new_dfs = {}
for event_type in df_final_matches['type'].unique():
  df_name = f"df_{event_type.replace(' ', '_').lower()}"
  new_dfs[df_name] = df_final_matches[df_final_matches['type'] == event_type].copy()

print("List of new dataframes:")
for df_name in new_dfs.keys():
    print(df_name)


List of new dataframes:
df_starting_xi
df_half_start
df_pass
df_ball_receipt*
df_carry
df_pressure
df_miscontrol
df_clearance
df_duel
df_foul_committed
df_foul_won
df_shot
df_goal_keeper
df_ball_recovery
df_block
df_interception
df_dribbled_past
df_dribble
df_dispossessed
df_50/50
df_injury_stoppage
df_player_off
df_player_on
df_half_end
df_substitution
df_tactical_shift
df_offside
df_shield
df_error
df_bad_behaviour
df_referee_ball-drop
df_own_goal_for
df_own_goal_against
     50_50 ball_receipt_outcome ball_recovery_offensive  \
1767   NaN                  NaN                     NaN   
1768   NaN                  NaN                     NaN   
1769   NaN                  NaN                     NaN   
1770   NaN                  NaN                     NaN   
1771   NaN                  NaN                     NaN   

     ball_recovery_recovery_failure block_deflection carry_end_location  \
1767                            NaN              NaN       [57.6, 38.1]   
1768             

**High Level**

In [ ]:
# Sum duration and shot_statsbomb_xg by match_id

match_analysis = df_final_matches.groupby('match_id').agg(
    duration=('duration', 'sum'),
    xg=('shot_statsbomb_xg', 'sum'),
    home_team_duration=('duration', lambda x: x[df_final_matches.loc[x.index, 'home_team_flag'] == 1].sum()),
    max_posession=('possession', 'max')
).reset_index()

# Calculate the shot_home_proportion after aggregation
match_analysis['home_team_duration'] = match_analysis['home_team_duration'] / match_analysis['duration']

match_analysis

,match_id,duration,xg,home_team_duration,max_posession
0,3753972,3351.219735,3.937206,0.428163,178
1,3753973,3043.789623,3.291737,0.580829,201
2,3753974,3478.407684,3.027560,0.390655,167
3,3753975,3480.275568,1.673016,0.433782,176
4,3753976,3717.852678,2.501524,0.611265,192
...,...,...,...,...,...
245,3877072,3567.143040,1.217027,0.713331,159
246,3877090,2833.906310,3.653438,0.514969,183
247,3877115,3714.606408,1.808099,0.580162,154
248,3877170,3046.840864,3.361517,0.643220,164


**shots**

In [ ]:
selected_columns = [

    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team_flag',
    'duration',
    'counterpress',
    'index',
    'x',
    'y',
    'play_pattern',
    'shot_body_part',
    'shot_first_time',
    'shot_outcome',
    'shot_technique',
    'shot_type',
    'shot_statsbomb_xg'
]
df_shot = new_dfs['df_shot']
existing_columns = [col for col in selected_columns if col in df_shot.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_shot = df_shot[existing_columns]
df_final_matches_selected_shot

,id,match_id,location,period,possession,possession_team,under_pressure,home_team_flag,duration,counterpress,index,x,y,play_pattern,shot_body_part,shot_first_time,shot_outcome,shot_technique,shot_type,shot_statsbomb_xg
3006,9fff7d2c-36b4-428a-aa3b-ef84a59df46c,3829437,"[113.9, 45.7]",1,5,Angers,NaN,0,0.687298,NaN,53,113.9,45.7,From Free Kick,Head,NaN,Goal,Normal,Open Play,0.160688
3007,9ccb8700-16b0-40d6-ace5-534b0530964d,3829437,"[112.7, 41.3]",1,12,Gazélec Ajaccio,True,1,0.271717,NaN,166,112.7,41.3,From Corner,Head,NaN,Off T,Normal,Open Play,0.154405
3008,621c9829-a59d-4355-a715-c43b7127946f,3829437,"[92.1, 33.2]",1,27,Angers,NaN,0,1.202479,NaN,455,92.1,33.2,From Free Kick,Right Foot,NaN,Off T,Normal,Free Kick,0.047601
3009,3f44f334-954c-41e2-9175-c3b7c5f1b838,3829437,"[110.2, 47.0]",1,40,Angers,NaN,0,0.719912,NaN,657,110.2,47.0,From Free Kick,Right Foot,True,Goal,Volley,Open Play,0.235270
3010,2aba2b2a-15e7-401b-97fd-7cd286147e8c,3829437,"[112.8, 39.2]",1,51,Gazélec Ajaccio,True,1,0.379872,NaN,808,112.8,39.2,From Throw In,Head,NaN,Off T,Normal,Open Play,0.150905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859984,5758038d-99f9-4f19-b4b3-121ac81cda3d,3754031,"[104.8, 29.8]",2,161,Manchester United,NaN,0,0.700667,NaN,2812,104.8,29.8,Regular Play,Right Foot,NaN,Saved,Normal,Open Play,0.082604
859985,a7cb2a8c-6399-45f7-81a9-a2724018fc4e,3754031,"[112.7, 36.8]",2,165,Sunderland,NaN,1,0.411687,NaN,2830,112.7,36.8,From Corner,Head,NaN,Saved,Normal,Open Play,0.074779
859986,04c4a741-8950-4c0a-991a-b7aa6eb0ed74,3754031,"[106.7, 25.9]",2,174,Manchester United,NaN,0,0.599526,NaN,2960,106.7,25.9,From Throw In,Right Foot,NaN,Saved,Normal,Open Play,0.049532
859987,41c75354-fd7e-4e84-b5b6-a0ccef06bb5f,3754031,"[107.7, 41.5]",2,178,Manchester United,NaN,0,0.182451,NaN,2996,107.7,41.5,Regular Play,Right Foot,True,Blocked,Half Volley,Open Play,0.093657


In [ ]:
# Join df_final_matches_selected_shot and df_final_matches on 'match_id' and 'possession'
df_merged = pd.merge(df_final_matches_selected_shot[['match_id', 'possession']],
                     df_final_matches[['match_id', 'possession', 'index']],
                     on=['match_id', 'possession'],
                     how='inner')

# Find the minimum index for each match_id and possession combination
df_min_index = df_merged.groupby(['match_id', 'possession'])['index'].min().reset_index()
df_min_index.rename(columns={'index': 'min_index'}, inplace=True)

# Join the minimum index back onto df_final_matches_selected_shot
df_final_matches_selected_shot = pd.merge(df_final_matches_selected_shot,
                                          df_min_index,
                                          on=['match_id', 'possession'],
                                          how='left')

print(df_final_matches_selected_shot.head())


                                     id  match_id       location  period  \
0  9fff7d2c-36b4-428a-aa3b-ef84a59df46c   3829437  [113.9, 45.7]       1   
1  9ccb8700-16b0-40d6-ace5-534b0530964d   3829437  [112.7, 41.3]       1   
2  621c9829-a59d-4355-a715-c43b7127946f   3829437   [92.1, 33.2]       1   
3  3f44f334-954c-41e2-9175-c3b7c5f1b838   3829437  [110.2, 47.0]       1   
4  2aba2b2a-15e7-401b-97fd-7cd286147e8c   3829437  [112.8, 39.2]       1   

   possession  possession_team under_pressure  home_team_flag  duration  \
0           5           Angers            NaN               0  0.687298   
1          12  Gazélec Ajaccio           True               1  0.271717   
2          27           Angers            NaN               0  1.202479   
3          40           Angers            NaN               0  0.719912   
4          51  Gazélec Ajaccio           True               1  0.379872   

  counterpress  ...      x     y    play_pattern shot_body_part  \
0          NaN  ...  113.

In [ ]:
# Reset the min_index_diff_flag to 0 before recalculating
df_final_matches_selected_shot['sequence length'] = df_final_matches_selected_shot['index'] - df_final_matches_selected_shot['min_index']

df_final_matches_selected_shot['first_shot_flag'] = 0

# Group by match_id and possession
grouped = df_final_matches_selected_shot.groupby(['match_id', 'possession'])

# Iterate through each group and set the flag for the minimum index_diff
for name, group in grouped:
  min_index_diff_idx = group['sequence length'].idxmin()
  df_final_matches_selected_shot.loc[min_index_diff_idx, 'first_shot_flag'] = 1

df_final_matches_selected_shot

,id,match_id,location,period,possession,possession_team,under_pressure,home_team_flag,duration,counterpress,...,play_pattern,shot_body_part,shot_first_time,shot_outcome,shot_technique,shot_type,shot_statsbomb_xg,min_index,sequence length,first_shot_flag
0,9fff7d2c-36b4-428a-aa3b-ef84a59df46c,3829437,"[113.9, 45.7]",1,5,Angers,NaN,0,0.687298,NaN,...,From Free Kick,Head,NaN,Goal,Normal,Open Play,0.160688,49,4,1
1,9ccb8700-16b0-40d6-ace5-534b0530964d,3829437,"[112.7, 41.3]",1,12,Gazélec Ajaccio,True,1,0.271717,NaN,...,From Corner,Head,NaN,Off T,Normal,Open Play,0.154405,158,8,1
2,621c9829-a59d-4355-a715-c43b7127946f,3829437,"[92.1, 33.2]",1,27,Angers,NaN,0,1.202479,NaN,...,From Free Kick,Right Foot,NaN,Off T,Normal,Free Kick,0.047601,455,0,1
3,3f44f334-954c-41e2-9175-c3b7c5f1b838,3829437,"[110.2, 47.0]",1,40,Angers,NaN,0,0.719912,NaN,...,From Free Kick,Right Foot,True,Goal,Volley,Open Play,0.235270,641,16,1
4,2aba2b2a-15e7-401b-97fd-7cd286147e8c,3829437,"[112.8, 39.2]",1,51,Gazélec Ajaccio,True,1,0.379872,NaN,...,From Throw In,Head,NaN,Off T,Normal,Open Play,0.150905,802,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6406,5758038d-99f9-4f19-b4b3-121ac81cda3d,3754031,"[104.8, 29.8]",2,161,Manchester United,NaN,0,0.700667,NaN,...,Regular Play,Right Foot,NaN,Saved,Normal,Open Play,0.082604,2781,31,1
6407,a7cb2a8c-6399-45f7-81a9-a2724018fc4e,3754031,"[112.7, 36.8]",2,165,Sunderland,NaN,1,0.411687,NaN,...,From Corner,Head,NaN,Saved,Normal,Open Play,0.074779,2828,2,1
6408,04c4a741-8950-4c0a-991a-b7aa6eb0ed74,3754031,"[106.7, 25.9]",2,174,Manchester United,NaN,0,0.599526,NaN,...,From Throw In,Right Foot,NaN,Saved,Normal,Open Play,0.049532,2898,62,1
6409,41c75354-fd7e-4e84-b5b6-a0ccef06bb5f,3754031,"[107.7, 41.5]",2,178,Manchester United,NaN,0,0.182451,NaN,...,Regular Play,Right Foot,True,Blocked,Half Volley,Open Play,0.093657,2982,14,1


In [ ]:

# Ensure df_final_matches_selected_shot is sorted by match_id, possession, and index
df_final_matches_selected_shot = df_final_matches_selected_shot.sort_values(by=['match_id', 'possession', 'index'])

# Initialize a list to store the counts for each shot
pass_counts = []

# Iterate through each row in df_final_matches_selected_shot
for index, row in df_final_matches_selected_shot.iterrows():
    match_id = row['match_id']
    shot_index = row['index']
    min_index = row['min_index']

    # Filter df_final_matches for the current match_id and the relevant index range
    relevant_events = df_final_matches[
        (df_final_matches['match_id'] == match_id) &
        (df_final_matches['index'] >= min_index) &
        (df_final_matches['index'] <= shot_index)
    ]

    # Count the number of rows where 'type' is 'Pass' in the filtered events
    count_passes = relevant_events[relevant_events['type'] == 'Pass'].shape[0]

    # Append the count to the list
    pass_counts.append(count_passes)

# Add the pass counts as a new column to df_final_matches_selected_shot
df_final_matches_selected_shot['passes_in_sequence'] = pass_counts

print(df_final_matches_selected_shot[['match_id', 'index', 'min_index', 'passes_in_sequence']].head())
print("\nFirst few rows of df_final_matches_selected_shot with 'passes_in_sequence' column:")
print(df_final_matches_selected_shot.head())

      match_id  index  min_index  passes_in_sequence
1266   3753972    143        126                   5
1267   3753972    366        346                   3
1268   3753972    415        402                   2
1269   3753972    435        417                   6
1270   3753972    440        437                   1

First few rows of df_final_matches_selected_shot with 'passes_in_sequence' column:
                                        id  match_id       location  period  \
1266  c0218ff0-32e8-46de-8ed4-a22e7ee4fd48   3753972  [101.8, 49.1]       1   
1267  d87ec24d-e894-4033-a1c3-85da30039ec6   3753972   [80.9, 43.8]       1   
1268  71a9e4f9-5755-4040-8756-18b4b76f77a0   3753972  [111.9, 39.9]       1   
1269  fcd48d58-d7a0-4617-b9d1-ee005e016484   3753972  [106.6, 28.7]       1   
1270  a301cbf2-b1f7-474f-92f6-5b480c87287d   3753972  [112.4, 48.6]       1   

      possession  possession_team under_pressure  home_team_flag  duration  \
1266           8  AFC Bournemouth           T

In [ ]:

df_final_matches_selected_shot = pd.merge(df_final_matches_selected_shot,
                                          df_selected_cols[['match_id', 'competition', 'season']],
                                          on='match_id',
                                          how='left')
print(df_final_matches_selected_shot.head())

                                     id  match_id       location  period  \
0  c0218ff0-32e8-46de-8ed4-a22e7ee4fd48   3753972  [101.8, 49.1]       1   
1  d87ec24d-e894-4033-a1c3-85da30039ec6   3753972   [80.9, 43.8]       1   
2  71a9e4f9-5755-4040-8756-18b4b76f77a0   3753972  [111.9, 39.9]       1   
3  fcd48d58-d7a0-4617-b9d1-ee005e016484   3753972  [106.6, 28.7]       1   
4  a301cbf2-b1f7-474f-92f6-5b480c87287d   3753972  [112.4, 48.6]       1   

   possession  possession_team under_pressure  home_team_flag  duration  \
0           8  AFC Bournemouth           True               0  1.287115   
1          24  AFC Bournemouth           True               0  0.048003   
2          26  AFC Bournemouth            NaN               0  0.745855   
3          27     Swansea City            NaN               1  0.285371   
4          28     Swansea City           True               1  0.284094   

  counterpress  ...  shot_outcome  shot_technique  shot_type  \
0          NaN  ...         

In [ ]:

# Group by match_id
match_analysis_shot = df_final_matches_selected_shot.groupby('match_id').agg(
    shot_count=('id', 'count'),
    shot_proportion_period_1=('period', lambda x: (x == 1).mean()),
    shot_proportion_home_team=('home_team_flag', 'mean'),
    shot_proportion_under_pressure=('under_pressure', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
    shot_duration_mean=('duration', 'mean'),
    shot_start_mean=('x', 'mean'),
    shot_start_proportion_x_lt_60=('x', lambda x: (x < 60).mean()), # Proportion of pass start x location when x is less than 60
    shot_start_y_between_20_60_proportion=('y', lambda y: ((y >= 20) & (y <= 60)).mean()), # Proportion of pass start y location when y is between 20 and 60
    shot_start_home=('x', lambda x: x[df_final_matches_selected_shot.loc[x.index, 'home_team_flag'] == 1].mean()),                       # Average y_dist
    shot_left_foot_proportion=('shot_body_part', lambda x: (x == 'Left Foot').mean()),
    shot_right_foot_proportion=('shot_body_part', lambda x: (x == 'Right Foot').mean()),
    shot_other_proportion=('shot_body_part', lambda x: ((x != 'Left Foot') & (x != 'Right Foot')).mean()),
    shot_first_shot_proportion=('first_shot_flag', 'mean'),
    shot_passes_in_sequence_mean=('passes_in_sequence', 'mean'),
    shot_sequence_length_mean=('sequence length', 'mean'),
    shot_type_proportion_open_play=('shot_type', lambda x: (x == 'Open Play').mean()),
    shot_play_pattern_proportion_regular_play=('play_pattern', lambda x: (x == 'Regular Play').mean()),
    shot_play_pattern_proportion_free_kick=('play_pattern', lambda x: (x == 'From Free Kick').mean()),
    shot_play_pattern_proportion_throw_in=('play_pattern', lambda x: (x == 'From Throw In').mean()),
    shot_play_pattern_proportion_corner=('play_pattern', lambda x: (x == 'From Corner').mean()),
    shot_play_pattern_proportion_counter=('play_pattern', lambda x: (x == 'From Counter').mean()),
    shot_technique_normal=('shot_technique', lambda x: (x == 'Normal').mean()),
    shot_first_time_proportion=('shot_first_time', lambda x: x.fillna(False).mean()),
    shot_on_target=('shot_outcome', lambda x: ((x == 'Goal') |  (x == 'Saved')).mean()),
    shot_blocked=('shot_outcome', lambda x: (x == 'Blocked').mean()),
    shot_off_target=('shot_outcome', lambda x: ((x != 'Goal')&(x != 'Blocked')& (x != 'Saved')).mean())
).reset_index()

match_analysis_shot

/tmp/ipython-input-31-2842444299.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  shot_proportion_under_pressure=('under_pressure', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
/tmp/ipython-input-31-2842444299.py:27: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  shot_first_time_proportion=('shot_first_time', lambda x: x.fillna(False).mean()),


,match_id,shot_count,shot_proportion_period_1,shot_proportion_home_team,shot_proportion_under_pressure,shot_duration_mean,shot_start_mean,shot_start_proportion_x_lt_60,shot_start_y_between_20_60_proportion,shot_start_home,...,shot_play_pattern_proportion_regular_play,shot_play_pattern_proportion_free_kick,shot_play_pattern_proportion_throw_in,shot_play_pattern_proportion_corner,shot_play_pattern_proportion_counter,shot_technique_normal,shot_first_time_proportion,shot_on_target,shot_blocked,shot_off_target
0,3753972,29,0.482759,0.413793,0.448276,0.562621,104.372414,0.000000,0.965517,105.091667,...,0.344828,0.103448,0.172414,0.137931,0.034483,0.724138,0.344828,0.275862,0.413793,0.310345
1,3753973,35,0.371429,0.628571,0.200000,0.686031,105.722857,0.000000,1.000000,106.050000,...,0.257143,0.285714,0.114286,0.228571,0.057143,0.771429,0.314286,0.285714,0.314286,0.400000
2,3753974,30,0.333333,0.333333,0.200000,0.609452,104.116667,0.000000,1.000000,106.460000,...,0.400000,0.233333,0.033333,0.200000,0.033333,0.866667,0.166667,0.266667,0.366667,0.366667
3,3753975,25,0.520000,0.400000,0.360000,0.802287,100.988000,0.000000,1.000000,100.630000,...,0.360000,0.200000,0.320000,0.120000,0.000000,0.760000,0.280000,0.400000,0.280000,0.320000
4,3753976,28,0.535714,0.607143,0.142857,0.705128,101.382143,0.035714,0.964286,103.900000,...,0.428571,0.071429,0.214286,0.178571,0.071429,0.821429,0.285714,0.250000,0.357143,0.392857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,3877072,20,0.350000,0.650000,0.200000,0.573039,100.610000,0.000000,0.950000,100.253846,...,0.300000,0.250000,0.250000,0.100000,0.050000,0.850000,0.200000,0.300000,0.450000,0.250000
246,3877090,31,0.451613,0.677419,0.322581,0.767976,103.751613,0.000000,1.000000,103.247619,...,0.419355,0.129032,0.129032,0.129032,0.064516,0.806452,0.258065,0.387097,0.193548,0.419355
247,3877115,21,0.523810,0.571429,0.142857,0.714368,105.323810,0.000000,1.000000,106.250000,...,0.476190,0.142857,0.285714,0.095238,0.000000,0.714286,0.380952,0.333333,0.238095,0.428571
248,3877170,33,0.424242,0.545455,0.272727,0.785117,103.433333,0.000000,1.000000,99.983333,...,0.303030,0.242424,0.121212,0.242424,0.060606,0.787879,0.333333,0.242424,0.151515,0.606061


**Passes**

In [ ]:
selected_columns = [

    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y',
    'pass_end_x',
    'pass_end_y',
    'pass_body_part',
    'pass_height',
    'pass_length'
]
# Check if the columns exist in the database
df_pass = new_dfs['df_pass']

existing_columns = [col for col in selected_columns if col in df_pass.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_pass = df_pass[existing_columns]

# Calculate the distance in x and y and add them as new columns
df_final_matches_selected_pass['x_dist'] = df_final_matches_selected_pass['pass_end_x'] - df_final_matches_selected_pass['x']
df_final_matches_selected_pass['y_dist'] = (df_final_matches_selected_pass['y'] - df_final_matches_selected_pass['pass_end_y']).abs()

df_final_matches_selected_pass

/tmp/ipython-input-32-3244172213.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_matches_selected_pass['x_dist'] = df_final_matches_selected_pass['pass_end_x'] - df_final_matches_selected_pass['x']
/tmp/ipython-input-32-3244172213.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_matches_selected_pass['y_dist'] = (df_final_matches_selected_pass['y'] - df_final_matches_selected_pass['pass_end_y']).abs()


,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y,pass_end_x,pass_end_y,pass_body_part,pass_height,pass_length,x_dist,y_dist
6,036647f4-f756-49eb-9198-f6c28db14983,3829437,"[60.0, 40.0]",1,2,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.683624,NaN,60.0,40.0,59.4,38.6,Right Foot,Ground Pass,1.523155,-0.6,1.4
7,0ec15dc2-c94c-49ac-b357-054a9449ad81,3829437,"[57.6, 38.1]",1,2,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,1.880896,NaN,57.6,38.1,40.4,11.5,Right Foot,Ground Pass,31.676489,-17.2,26.6
8,a801cb7c-cb42-4b3b-92e7-657d5051a581,3829437,"[39.6, 9.5]",1,2,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,1.293551,NaN,39.6,9.5,39.6,20.5,Left Foot,Ground Pass,11.000000,0.0,11.0
9,315e8ead-4592-4834-b882-d71910137151,3829437,"[43.0, 15.9]",1,2,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,1.052955,NaN,43.0,15.9,33.4,4.7,Right Foot,Ground Pass,14.751271,-9.6,11.2
10,896cc6c0-43c0-4602-9a7b-0e330de8fd6f,3829437,"[85.3, 80.0]",1,3,Angers,NaN,Gazélec Ajaccio,0,1.545203,NaN,85.3,80.0,108.6,61.5,NaN,High Pass,29.751303,23.3,18.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857913,0b735cfd-9ba9-43fa-a124-49ab0136afb1,3754031,"[80.0, 11.1]",2,193,Manchester United,NaN,Sunderland,0,1.293573,NaN,80.0,11.1,68.2,41.4,Left Foot,Ground Pass,32.516613,-11.8,30.3
857914,be83837f-f8e1-4049-9331-1e9ac20d13a3,3754031,"[73.2, 40.9]",2,193,Manchester United,NaN,Sunderland,0,0.673641,NaN,73.2,40.9,91.5,45.3,Right Foot,Low Pass,18.821530,18.3,4.4
857915,7f950ea0-21d3-426b-aec7-e8aade8043f2,3754031,"[86.8, 41.9]",2,193,Manchester United,NaN,Sunderland,0,1.780535,NaN,86.8,41.9,104.5,48.2,Left Foot,High Pass,18.787762,17.7,6.3
857916,b4ccb5d5-10a3-4444-a7ea-721e7a6fea0c,3754031,"[15.3, 31.9]",2,193,Manchester United,NaN,Sunderland,0,1.884939,NaN,15.3,31.9,34.9,38.7,Head,High Pass,20.746084,19.6,6.8


In [ ]:

df_final_matches_selected_pass['short pass'] = (df_final_matches_selected_pass['pass_length'] < 10).astype(int)
df_final_matches_selected_pass['medium pass'] = ((df_final_matches_selected_pass['pass_length'] >= 10) & (df_final_matches_selected_pass['pass_length'] < 30)).astype(int)
df_final_matches_selected_pass['long pass'] = (df_final_matches_selected_pass['pass_length'] >= 30).astype(int)



/tmp/ipython-input-33-4175800676.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_matches_selected_pass['short pass'] = (df_final_matches_selected_pass['pass_length'] < 10).astype(int)
/tmp/ipython-input-33-4175800676.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_matches_selected_pass['medium pass'] = ((df_final_matches_selected_pass['pass_length'] >= 10) & (df_final_matches_selected_pass['pass_length'] < 30)).astype(int)
/tmp/ipython-input-33-4175800676.py:5: SettingWithCopyWarni

In [ ]:

# Group by match_id
match_analysis_pass = df_final_matches_selected_pass.groupby('match_id').agg(
    pass_count=('id', 'count'),
    pass_proportion_period_1=('period', lambda x: (x == 1).mean()),
    pass_proportion_home_team=('home_team_flag', 'mean'),
    pass_proportion_under_pressure=('under_pressure', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
    pass_duration_mean=('duration', 'mean'),
    pass_counterpress_rate=('counterpress', lambda x: x.fillna(False).mean()),
    pass_start_mean=('x', 'mean'),
    pass_start_proportion_x_lt_60=('x', lambda x: (x < 60).mean()), # Proportion of pass start x location when x is less than 60
    pass_start_y_between_20_60_proportion=('y', lambda y: ((y >= 20) & (y <= 60)).mean()), # Proportion of pass start y location when y is between 20 and 60
    pass_start_home=('x', lambda x: x[df_final_matches_selected_pass.loc[x.index, 'home_team_flag'] == 1].mean()),
    pass_proportion_x_dist_lt_0=('x_dist', lambda x: (x < 0).mean()),  # Proportion of x_dist < 0
    pass_x_dist_lt_0=('x_dist', lambda x: x[x < 0].mean()),      # Count of x_dist < 0
    pass_average_x_dist_gt_0=('x_dist', lambda x: x[x > 0].mean()),  # Average x_dist > 0
    pass_average_y_dist=('y_dist', 'mean'),                          # Average y_dist
    pass_average_x_dist_lt_0_home=('x_dist', lambda x: x[(df_final_matches_selected_pass.loc[x.index, 'home_team_flag'] == 1) & (x < 0)].mean()),# Average x_dist < 0 by home_team_flag
    pass_average_x_dist_gt_0_home=('x_dist', lambda x: x[(df_final_matches_selected_pass.loc[x.index, 'home_team_flag'] == 1) & (x > 0)].mean()),# Average x_dist < 0 by home_team_flag
    pass_left_foot_proportion=('pass_body_part', lambda x: (x == 'Left Foot').mean()),
    pass_right_foot_proportion=('pass_body_part', lambda x: (x == 'Right Foot').mean()),
    pass_other_proportion=('pass_body_part', lambda x: ((x != 'Left Foot') & (x != 'Right Foot')).mean()),
    pass_pass_height_proportion_ground=('pass_height', lambda x: (x == 'Ground Pass').mean()),
    pass_pass_height_proportion_low=('pass_height', lambda x: (x == 'Low Pass').mean()),
    pass_pass_height_proportion_high=('pass_height', lambda x: (x == 'High Pass').mean()),
    pass_short_pass_proportion=('short pass', 'mean'),
    pass_medium_pass_proportion=('medium pass', 'mean'),
    pass_long_pass_proportion=('long pass', 'mean')
).reset_index()

match_analysis_pass

/tmp/ipython-input-34-4220353741.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pass_proportion_under_pressure=('under_pressure', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
/tmp/ipython-input-34-4220353741.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pass_counterpress_rate=('counterpress', lambda x: x.fillna(False).mean()),


,match_id,pass_count,pass_proportion_period_1,pass_proportion_home_team,pass_proportion_under_pressure,pass_duration_mean,pass_counterpress_rate,pass_start_mean,pass_start_proportion_x_lt_60,pass_start_y_between_20_60_proportion,...,pass_average_x_dist_gt_0_home,pass_left_foot_proportion,pass_right_foot_proportion,pass_other_proportion,pass_pass_height_proportion_ground,pass_pass_height_proportion_low,pass_pass_height_proportion_high,pass_short_pass_proportion,pass_medium_pass_proportion,pass_long_pass_proportion
0,3753972,1032,0.485465,0.427326,0.153101,1.693873,0.000000,59.117829,0.521318,0.449612,...,19.261314,0.319767,0.541667,0.138566,0.603682,0.170543,0.225775,0.215116,0.584302,0.200581
1,3753973,963,0.526480,0.599169,0.156802,1.660979,0.003115,57.179128,0.578401,0.553479,...,15.676738,0.185877,0.686397,0.127726,0.624091,0.147456,0.228453,0.164071,0.613707,0.222222
2,3753974,976,0.524590,0.413934,0.180328,1.610859,0.002049,61.671619,0.464139,0.498975,...,17.710247,0.313525,0.580943,0.105533,0.628074,0.182377,0.189549,0.206967,0.611680,0.181352
3,3753975,1009,0.500496,0.423191,0.225966,1.541302,0.004955,59.503469,0.492567,0.471754,...,20.547368,0.308226,0.554014,0.137760,0.645193,0.167493,0.187314,0.155600,0.628345,0.216056
4,3753976,1073,0.518173,0.630941,0.165890,1.511659,0.004660,60.758993,0.471575,0.467847,...,14.747368,0.283318,0.616030,0.100652,0.693383,0.146319,0.160298,0.181733,0.634669,0.183597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,3877072,1089,0.515152,0.705234,0.141414,1.582755,0.001837,60.440955,0.459137,0.569330,...,10.585808,0.326905,0.600551,0.072544,0.813590,0.075298,0.111111,0.197429,0.664830,0.137741
246,3877090,901,0.520533,0.470588,0.176471,1.521949,0.001110,57.223418,0.532741,0.488346,...,15.127715,0.302997,0.605993,0.091010,0.712542,0.109878,0.177580,0.174251,0.620422,0.205327
247,3877115,1204,0.509136,0.598007,0.125415,1.438388,0.003322,56.917774,0.544020,0.563123,...,11.851733,0.295681,0.653654,0.050664,0.817276,0.078073,0.104651,0.189369,0.694352,0.116279
248,3877170,942,0.538217,0.627389,0.148620,1.457409,0.005308,58.608599,0.507431,0.542463,...,13.159302,0.324841,0.594480,0.080679,0.719745,0.128450,0.151805,0.199575,0.643312,0.157113


**Carries** - reviewed

In [ ]:
selected_columns = [

    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y',
    'carry_end_x',
    'carry_end_y'
]
# Check if the columns exist in the database
df_carry = new_dfs['df_carry']

existing_columns = [col for col in selected_columns if col in df_carry.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_carry = df_carry[existing_columns]

# Calculate the distance in x and y and add them as new columns
df_final_matches_selected_carry['x_dist'] = df_final_matches_selected_carry['carry_end_x'] - df_final_matches_selected_carry['x']
df_final_matches_selected_carry['y_dist'] = (df_final_matches_selected_carry['y'] - df_final_matches_selected_carry['carry_end_y']).abs()

df_final_matches_selected_carry

/tmp/ipython-input-35-1735281139.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_matches_selected_carry['x_dist'] = df_final_matches_selected_carry['carry_end_x'] - df_final_matches_selected_carry['x']
/tmp/ipython-input-35-1735281139.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_matches_selected_carry['y_dist'] = (df_final_matches_selected_carry['y'] - df_final_matches_selected_carry['carry_end_y']).abs()


,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y,carry_end_x,carry_end_y,x_dist,y_dist
1767,fc1468a4-ce5a-485f-8763-1e58697edc33,3829437,"[59.4, 38.6]",1,2,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.821781,NaN,59.4,38.6,57.6,38.1,-1.8,0.5
1768,eae8fb0d-203b-4657-a25d-b9b6148e91b3,3829437,"[40.4, 11.5]",1,2,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,2.727979,NaN,40.4,11.5,39.6,9.5,-0.8,2.0
1769,4da962da-4ab2-495a-a40c-535785932d4b,3829437,"[39.6, 20.5]",1,2,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.792221,NaN,39.6,20.5,43.0,15.9,3.4,4.6
1770,9ce2d94f-8408-4ee4-a5b7-8fe5634ce099,3829437,"[33.4, 4.7]",1,2,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.114966,NaN,33.4,4.7,33.4,4.7,0.0,0.0
1771,045c4fe8-6190-43f0-9332-31ae1e9796cc,3829437,"[91.0, 74.2]",1,3,Angers,NaN,Gazélec Ajaccio,0,0.031913,NaN,91.0,74.2,90.1,73.9,-0.9,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859334,9906771c-5714-4c3f-82fc-f726bb0278bb,3754031,"[37.0, 17.8]",2,192,Manchester United,NaN,Sunderland,0,8.409728,NaN,37.0,17.8,78.7,8.4,41.7,9.4
859335,7ddf932b-b663-4fbc-9e5c-6149d69b34de,3754031,"[78.7, 8.4]",2,192,Manchester United,True,Sunderland,0,3.499934,NaN,78.7,8.4,108.6,23.4,29.9,15.0
859336,9d8f063b-8840-4b6c-8594-a1f032a9cff8,3754031,"[95.4, 8.8]",2,193,Manchester United,True,Sunderland,0,3.594468,NaN,95.4,8.8,89.9,11.1,-5.5,2.3
859337,11fe180a-cb0f-4b2e-94d6-394353a75d46,3754031,"[79.7, 10.9]",2,193,Manchester United,NaN,Sunderland,0,0.802244,NaN,79.7,10.9,80.0,11.1,0.3,0.2


In [ ]:

# Group by match_id
match_analysis_carry = df_final_matches_selected_carry.groupby('match_id').agg(
    carry_count=('id', 'count'),
    carry_proportion_period_1=('period', lambda x: (x == 1).mean()),
    carry_proportion_home_team=('home_team_flag', 'mean'),
    carry_proportion_under_pressure=('under_pressure', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
    carry_duration_mean=('duration', 'mean'),
    carry_counterpress_rate=('counterpress', lambda x: x.fillna(False).mean()),
    carry_start_mean=('x', 'mean'),
    carry_start_proportion_x_lt_60=('x', lambda x: (x < 60).mean()), # Proportion of carry start x location when x is less than 60
    carry_start_y_between_20_60_proportion=('y', lambda y: ((y >= 20) & (y <= 60)).mean()), # Proportion of carry start y location when y is between 20 and 60
    carry_start_home=('x', lambda x: x[df_final_matches_selected_carry.loc[x.index, 'home_team_flag'] == 1].mean()),
    carry_proportion_x_dist_lt_0=('x_dist', lambda x: (x < 0).mean()),  # Proportion of x_dist < 0
    carry_x_dist_lt_0=('x_dist', lambda x: x[x < 0].mean()),      # Count of x_dist < 0
    carry_average_x_dist_gt_0=('x_dist', lambda x: x[x > 0].mean()),  # Average x_dist > 0
    carry_average_y_dist=('y_dist', 'mean'),                          # Average y_dist
    carry_average_x_dist_lt_0_home=('x_dist', lambda x: x[(df_final_matches_selected_carry.loc[x.index, 'home_team_flag'] == 1) & (x < 0)].mean()),# Average x_dist < 0 by home_team_flag
    carry_average_x_dist_gt_0_home=('x_dist', lambda x: x[(df_final_matches_selected_carry.loc[x.index, 'home_team_flag'] == 1) & (x > 0)].mean())# Average x_dist < 0 by home_team_flag
).reset_index()

match_analysis_carry

/tmp/ipython-input-36-4165463729.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  carry_proportion_under_pressure=('under_pressure', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
/tmp/ipython-input-36-4165463729.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  carry_counterpress_rate=('counterpress', lambda x: x.fillna(False).mean()),


,match_id,carry_count,carry_proportion_period_1,carry_proportion_home_team,carry_proportion_under_pressure,carry_duration_mean,carry_counterpress_rate,carry_start_mean,carry_start_proportion_x_lt_60,carry_start_y_between_20_60_proportion,carry_start_home,carry_proportion_x_dist_lt_0,carry_x_dist_lt_0,carry_average_x_dist_gt_0,carry_average_y_dist,carry_average_x_dist_lt_0_home,carry_average_x_dist_gt_0_home
0,3753972,739,0.479026,0.400541,0.353180,1.770585,0.0,59.736942,0.511502,0.446549,56.764865,0.320704,-2.652321,6.435516,3.459540,-2.493814,6.180380
1,3753973,656,0.548780,0.597561,0.381098,1.694497,0.0,60.611433,0.532012,0.513720,61.880612,0.292683,-2.476042,5.938120,3.138872,-2.505357,5.843777
2,3753974,782,0.487212,0.372123,0.374680,1.946001,0.0,63.047826,0.443734,0.502558,61.300000,0.251918,-2.890355,6.667490,3.776854,-2.957647,6.894886
3,3753975,797,0.504391,0.402760,0.376412,1.913507,0.0,61.227604,0.462986,0.475533,60.755763,0.297365,-2.837553,7.284938,3.193350,-2.795690,6.921854
4,3753976,886,0.531603,0.612867,0.386005,1.829389,0.0,61.736682,0.451467,0.492099,61.950829,0.272009,-3.117012,6.126078,3.477201,-3.508784,6.137224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,3877072,845,0.508876,0.727811,0.246154,1.898507,0.0,61.395030,0.463905,0.566864,66.484390,0.231953,-2.329082,5.467757,2.457278,-2.409375,5.567692
246,3877090,709,0.506347,0.475317,0.375176,1.670089,0.0,60.349929,0.493653,0.485190,62.345104,0.280677,-2.586935,5.577494,3.013822,-2.546237,6.396517
247,3877115,1010,0.509901,0.555446,0.264356,1.736993,0.0,58.388119,0.530693,0.550495,62.054724,0.294059,-2.713131,4.861419,2.812376,-2.706329,4.851471
248,3877170,804,0.526119,0.621891,0.333333,1.790206,0.0,61.235945,0.466418,0.548507,62.864800,0.247512,-2.423116,5.778151,2.906468,-2.259350,6.076744


**Pressure** - reviewed

In [ ]:
selected_columns = [

    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y'
]
df_pressure = new_dfs['df_pressure']
existing_columns = [col for col in selected_columns if col in df_pressure.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_pressure = df_pressure[existing_columns]
df_final_matches_selected_pressure

,id,match_id,location,period,possession,possession_team,under_pressure,home_team_flag,duration,counterpress,x,y
2448,6e101170-89f5-4db3-98d6-7d3b2ad2f327,3829437,"[74.2, 71.2]",1,2,Gazélec Ajaccio,NaN,1,0.704793,NaN,74.2,71.2
2449,97f05763-cae3-4306-97ca-106ae0fcdcfd,3829437,"[24.7, 14.7]",1,3,Angers,NaN,0,0.382236,NaN,24.7,14.7
2450,8645dae6-8ef4-4bef-a40c-5de18a50dc22,3829437,"[106.3, 32.4]",1,3,Angers,NaN,0,0.860765,NaN,106.3,32.4
2451,495e6bd9-f0ad-4589-a8ed-cdeb8dd6f0bc,3829437,"[25.4, 66.0]",1,4,Angers,NaN,0,1.410999,NaN,25.4,66.0
2452,8698d215-bfe5-48a3-a7fd-43a7c74348c6,3829437,"[31.6, 64.7]",1,4,Angers,NaN,0,1.055789,NaN,31.6,64.7
...,...,...,...,...,...,...,...,...,...,...,...,...
859803,d7e010f1-a91b-44d9-85a9-1853dae7f7d1,3754031,"[31.5, 72.2]",2,189,Sunderland,NaN,1,0.590502,NaN,31.5,72.2
859804,2b29e4cb-f916-4f5c-9759-48b3b28c1c43,3754031,"[95.7, 10.5]",2,189,Sunderland,NaN,1,0.852373,NaN,95.7,10.5
859805,1615231a-82e5-486e-a814-2fb0fedf373d,3754031,"[18.9, 77.4]",2,191,Sunderland,NaN,1,0.341686,NaN,18.9,77.4
859806,2c87ca3b-f137-4140-93f8-1169aa8709dd,3754031,"[25.7, 71.7]",2,191,Sunderland,NaN,1,4.381108,NaN,25.7,71.7


In [ ]:
match_analysis_pressure = df_final_matches_selected_pressure.groupby('match_id').agg(
    pressure_count=('id', 'count'),
    pressure_proportion_period_1=('period', lambda x: (x == 1).mean()),
    pressure_proportion_home_team=('home_team_flag', 'mean'),
    pressure_duration_mean=('duration', 'mean'),
    pressure_counterpress_rate=('counterpress', lambda x: x.fillna(False).mean()),
    pressure_start_mean=('x', 'mean'),
    pressure_start_proportion_x_lt_60=('x', lambda x: (x < 60).mean()), # Proportion of carry start x location when x is less than 60
    pressure_start_y_between_20_60_proportion=('y', lambda y: ((y >= 20) & (y <= 60)).mean()) # Proportion of carry start y location when y is between 20 and 60
).reset_index()

match_analysis_pressure

/tmp/ipython-input-38-2132458553.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pressure_counterpress_rate=('counterpress', lambda x: x.fillna(False).mean()),


,match_id,pressure_count,pressure_proportion_period_1,pressure_proportion_home_team,pressure_duration_mean,pressure_counterpress_rate,pressure_start_mean,pressure_start_proportion_x_lt_60,pressure_start_y_between_20_60_proportion
0,3753972,368,0.478261,0.388587,0.751877,0.198370,52.436957,0.584239,0.459239
1,3753973,308,0.542208,0.493506,0.981961,0.217532,55.971429,0.555195,0.506494
2,3753974,357,0.456583,0.431373,1.013559,0.221289,51.666947,0.619048,0.585434
3,3753975,396,0.517677,0.512626,0.951885,0.171717,54.561869,0.603535,0.512626
4,3753976,370,0.510811,0.510811,1.230434,0.243243,56.607297,0.575676,0.456757
...,...,...,...,...,...,...,...,...,...
245,3877072,285,0.508772,0.617544,0.799381,0.207018,54.507719,0.571930,0.659649
246,3877090,321,0.495327,0.470405,0.777942,0.196262,56.985981,0.560748,0.548287
247,3877115,333,0.522523,0.489489,0.631509,0.171171,59.040841,0.498498,0.624625
248,3877170,316,0.500000,0.582278,0.660530,0.205696,52.687975,0.617089,0.617089


**miscontrol** - reviewed

In [ ]:
selected_columns = [

    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y'
]
# Check if the columns exist in the database
df_miscontrol = new_dfs['df_miscontrol']

existing_columns = [col for col in selected_columns if col in df_miscontrol.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_miscontrol = df_miscontrol[existing_columns]
df_final_matches_selected_miscontrol

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y
2768,eba07cdc-b74f-4f09-a8e0-1e05d456a2f9,3829437,"[33.4, 4.7]",1,2,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,33.4,4.7
2769,971dea78-d7ca-4b04-8f63-c12dfdb1db70,3829437,"[75.7, 26.1]",1,16,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,75.7,26.1
2770,c71ace11-d307-40c0-ad6c-76eca2b3cd53,3829437,"[79.0, 75.0]",1,17,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,79.0,75.0
2771,d1d8f6fe-271b-42e4-8578-26a389ce32ed,3829437,"[77.3, 21.1]",1,28,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,77.3,21.1
2772,20824ca7-ab6f-41ee-a0c4-bd6e8377d1c4,3829437,"[66.7, 11.6]",1,34,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,66.7,11.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
860193,2f99af62-d94e-4868-ba72-530272dc6423,3754031,"[94.3, 55.0]",2,111,Manchester United,NaN,Sunderland,0,0.0,NaN,94.3,55.0
860194,b4af14cb-f747-4414-a77f-ac4236d5bdb2,3754031,"[97.5, 22.6]",2,150,Manchester United,NaN,Sunderland,0,0.0,NaN,97.5,22.6
860195,6eee3bdb-e788-44f0-b963-943e0218ff24,3754031,"[91.8, 57.7]",2,155,Sunderland,True,Sunderland,1,0.0,NaN,91.8,57.7
860196,fa0042e2-31a8-4f26-8b19-b3491e564a39,3754031,"[69.3, 5.6]",2,160,Sunderland,True,Sunderland,1,0.0,NaN,69.3,5.6


In [ ]:

# Group by match_id
match_analysis_miscontrol = df_final_matches_selected_miscontrol.groupby('match_id').agg(
    miscontrol_count=('id', 'count'),
    miscontrol_proportion_period_1=('period', lambda x: (x == 1).mean()),
    miscontrol_proportion_home_team=('home_team_flag', 'mean'),
    miscontrol_proportion_under_pressure=('under_pressure', lambda x: x.fillna(False).mean()),
    miscontrol_start_mean=('x', 'mean'),
    miscontrol_start_proportion_x_lt_60=('x', lambda x: (x < 60).mean()), # Proportion of carry start x location when x is less than 60
    miscontrol_start_y_between_20_60_proportion=('y', lambda y: ((y >= 20) & (y <= 60)).mean()) # Proportion of carry start y location when y is between 20 and 60
).reset_index()

match_analysis_miscontrol

/tmp/ipython-input-40-2137105281.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  miscontrol_proportion_under_pressure=('under_pressure', lambda x: x.fillna(False).mean()),


,match_id,miscontrol_count,miscontrol_proportion_period_1,miscontrol_proportion_home_team,miscontrol_proportion_under_pressure,miscontrol_start_mean,miscontrol_start_proportion_x_lt_60,miscontrol_start_y_between_20_60_proportion
0,3753972,43,0.488372,0.465116,0.325581,67.941860,0.348837,0.395349
1,3753973,25,0.480000,0.520000,0.320000,78.224000,0.160000,0.400000
2,3753974,15,0.333333,0.400000,0.533333,86.400000,0.133333,1.000000
3,3753975,32,0.531250,0.500000,0.312500,79.468750,0.187500,0.468750
4,3753976,26,0.423077,0.692308,0.269231,78.400000,0.230769,0.538462
...,...,...,...,...,...,...,...,...
245,3877072,17,0.352941,0.588235,0.235294,84.200000,0.235294,0.705882
246,3877090,34,0.558824,0.382353,0.147059,71.129412,0.441176,0.441176
247,3877115,24,0.541667,0.416667,0.291667,69.533333,0.416667,0.458333
248,3877170,32,0.468750,0.406250,0.281250,72.040625,0.281250,0.718750


**Dribble** - reviewed

In [ ]:
selected_columns = [

    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'dribble_outcome',
    'counterpress',
    'x',
    'y'
]
# Check if the columns exist in the database
df_dribble = new_dfs['df_dribble']

existing_columns = [col for col in selected_columns if col in df_carry.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_dribble = df_dribble[existing_columns]
df_final_matches_selected_dribble

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,dribble_outcome,counterpress,x,y
3226,491dadd2-6417-4899-bded-72d8b9694ca2,3829437,"[68.5, 66.2]",1,10,Angers,True,Gazélec Ajaccio,0,0.0,Complete,NaN,68.5,66.2
3227,4b13e58e-8c1f-46c0-bdf8-2430feb9e8ce,3829437,"[88.7, 5.0]",1,13,Angers,True,Gazélec Ajaccio,0,0.0,Complete,NaN,88.7,5.0
3228,458dcdf6-f2c2-4003-8dca-7af2561d54ef,3829437,"[75.4, 71.7]",1,47,Angers,True,Gazélec Ajaccio,0,0.0,Complete,NaN,75.4,71.7
3229,0c6306ca-762a-4a7d-8a96-84c04e58b82d,3829437,"[111.3, 60.5]",1,65,Angers,True,Gazélec Ajaccio,0,0.0,Incomplete,NaN,111.3,60.5
3230,6931f579-8d56-407c-a697-48fdbb97b4f1,3829437,"[38.7, 7.5]",1,72,Angers,True,Gazélec Ajaccio,0,0.0,Complete,NaN,38.7,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860115,783998f8-e881-496a-bc77-ef38a519378d,3754031,"[41.7, 28.8]",2,182,Sunderland,True,Sunderland,1,0.0,Complete,NaN,41.7,28.8
860116,4ded8191-5a24-463b-a62d-99552497c48e,3754031,"[83.9, 2.4]",2,186,Sunderland,True,Sunderland,1,0.0,Incomplete,NaN,83.9,2.4
860117,610309f0-d4dd-4b54-b084-ff797b14574f,3754031,"[59.9, 1.7]",2,188,Sunderland,True,Sunderland,1,0.0,Complete,NaN,59.9,1.7
860118,74d9036a-f018-4994-951a-234f232175d7,3754031,"[98.5, 1.4]",2,191,Sunderland,True,Sunderland,1,0.0,Incomplete,NaN,98.5,1.4


In [ ]:
# Group by match_id
match_analysis_dribble = df_final_matches_selected_dribble.groupby('match_id').agg(
    dribble_count=('id', 'count'),
    dribble_proportion_period_1=('period', lambda x: (x == 1).mean()),
    dribble_proportion_home_team=('home_team_flag', 'mean'), # Handle potential NaNs in under_pressure
    dribble_success_rate=('dribble_outcome', lambda x: (x == 'Complete').mean()),
    dribble_start_mean=('x', 'mean'),
    dribble_start_proportion_x_lt_60=('x', lambda x: (x < 60).mean()), # Proportion of carry start x location when x is less than 60
    dribble_start_y_between_20_60_proportion=('y', lambda y: ((y >= 20) & (y <= 60)).mean()) # Proportion of carry start y location when y is between 20 and 60
).reset_index()

match_analysis_dribble

,match_id,dribble_count,dribble_proportion_period_1,dribble_proportion_home_team,dribble_success_rate,dribble_start_mean,dribble_start_proportion_x_lt_60,dribble_start_y_between_20_60_proportion
0,3753972,31,0.322581,0.419355,0.548387,75.367742,0.290323,0.451613
1,3753973,45,0.666667,0.600000,0.555556,74.993333,0.355556,0.400000
2,3753974,64,0.343750,0.312500,0.593750,76.285938,0.296875,0.562500
3,3753975,42,0.380952,0.523810,0.428571,74.376190,0.261905,0.571429
4,3753976,42,0.380952,0.619048,0.666667,74.907143,0.309524,0.428571
...,...,...,...,...,...,...,...,...
245,3877072,26,0.423077,0.346154,0.615385,62.434615,0.500000,0.576923
246,3877090,42,0.500000,0.476190,0.595238,68.080952,0.380952,0.571429
247,3877115,35,0.485714,0.457143,0.514286,76.211429,0.285714,0.628571
248,3877170,38,0.368421,0.578947,0.578947,78.544737,0.210526,0.631579


**df_duel** - reviewed

In [ ]:
selected_columns = [

    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y',
    'duel_type'
]
# Check if the columns exist in the database
df_duel = new_dfs['df_duel']

existing_columns = [col for col in selected_columns if col in df_duel.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_duel = df_duel[existing_columns]
df_final_matches_selected_duel

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y,duel_type
2851,6e10c34c-43ec-4765-9331-26bc21c89651,3829437,"[105.1, 74.2]",1,3,Angers,True,Gazélec Ajaccio,0,0.0,NaN,105.1,74.2,Aerial Lost
2852,e677bf0d-c91d-4c92-b972-efaee5fd0190,3829437,"[29.5, 40.5]",1,3,Angers,True,Gazélec Ajaccio,0,0.0,NaN,29.5,40.5,Aerial Lost
2853,0a943617-e077-40a4-87cb-03aa92a1ab56,3829437,"[77.6, 6.7]",1,3,Angers,True,Gazélec Ajaccio,0,0.0,NaN,77.6,6.7,Aerial Lost
2854,7da9f5e3-2a3d-441a-b83c-58aae6edb108,3829437,"[75.6, 16.2]",1,6,Gazélec Ajaccio,True,Gazélec Ajaccio,1,0.0,NaN,75.6,16.2,Aerial Lost
2855,0144ae8f-1676-474c-a545-0c816546d9e7,3829437,"[116.0, 47.3]",1,12,Gazélec Ajaccio,True,Gazélec Ajaccio,1,0.0,NaN,116.0,47.3,Aerial Lost
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859904,e3825898-2bb2-4c1a-bae2-06f2e0ff4c94,3754031,"[99.6, 7.9]",2,189,Sunderland,True,Sunderland,1,0.0,NaN,99.6,7.9,Aerial Lost
859905,2ab53a46-a431-4fd8-99d5-00267dfb6017,3754031,"[64.3, 43.0]",2,190,Manchester United,True,Sunderland,0,0.0,NaN,64.3,43.0,Aerial Lost
859906,e368b57a-ebf8-405f-862b-5b9829d4a31c,3754031,"[65.4, 35.1]",2,190,Manchester United,True,Sunderland,0,0.0,NaN,65.4,35.1,Aerial Lost
859907,4e8e0c86-a402-43d8-9ec3-167786ef5ad4,3754031,"[21.6, 78.7]",2,191,Sunderland,True,Sunderland,1,0.0,NaN,21.6,78.7,Tackle


In [ ]:
# Group by match_id
match_analysis_duel = df_final_matches_selected_duel.groupby('match_id').agg(
    duel_count=('id', 'count'),
    duel_proportion_period_1=('period', lambda x: (x == 1).mean()),
    duel_proportion_away_team=('home_team_flag', 'mean'),
    duel_counterpress_rate=('counterpress', lambda x: x.fillna(False).mean()),
    duel_start_mean=('x', 'mean'),
    duel_proportion_Tackle=('duel_type', lambda x: (x == 'Tackle').mean()), #Tackle and Header are the only options
    duel_start_proportion_x_lt_60=('x', lambda x: (x < 60).mean()), # Proportion of carry start x location when x is less than 60
    duel_start_y_between_20_60_proportion=('y', lambda y: ((y >= 20) & (y <= 60)).mean()) # Proportion of carry start y location when y is between 20 and 60
).reset_index()

match_analysis_duel

/tmp/ipython-input-44-1019063939.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  duel_counterpress_rate=('counterpress', lambda x: x.fillna(False).mean()),


,match_id,duel_count,duel_proportion_period_1,duel_proportion_away_team,duel_counterpress_rate,duel_start_mean,duel_proportion_Tackle,duel_start_proportion_x_lt_60,duel_start_y_between_20_60_proportion
0,3753972,57,0.491228,0.578947,0.263158,58.336842,0.508772,0.473684,0.526316
1,3753973,94,0.510638,0.404255,0.191489,50.561702,0.457447,0.638298,0.531915
2,3753974,79,0.468354,0.493671,0.316456,51.483544,0.582278,0.607595,0.582278
3,3753975,89,0.573034,0.606742,0.269663,55.649438,0.528090,0.539326,0.449438
4,3753976,71,0.492958,0.676056,0.436620,51.605634,0.661972,0.633803,0.507042
...,...,...,...,...,...,...,...,...,...
245,3877072,35,0.514286,0.428571,0.342857,48.088571,0.685714,0.685714,0.542857
246,3877090,59,0.508475,0.508475,0.389831,52.954237,0.661017,0.576271,0.593220
247,3877115,46,0.586957,0.456522,0.434783,56.036957,0.891304,0.543478,0.586957
248,3877170,58,0.396552,0.517241,0.379310,48.055172,0.637931,0.637931,0.672414


**df_interception** - reviewed



In [ ]:
selected_columns = [

    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y'
]
# Check if the columns exist in the database
df_interception = new_dfs['df_interception']

existing_columns = [col for col in selected_columns if col in df_interception.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_interception = df_interception[existing_columns]
df_final_matches_selected_interception

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y
3190,1340b592-1fa1-4496-a82e-badf8b6840fa,3829437,"[67.9, 61.3]",1,10,Angers,True,Gazélec Ajaccio,0,0.0,True,67.9,61.3
3191,5082c647-3995-4531-a92d-9b50b5168b5e,3829437,"[23.1, 40.5]",1,10,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,23.1,40.5
3192,c329353a-f85e-4c1c-afd0-968a0d7cf43b,3829437,"[68.1, 9.6]",1,22,Angers,NaN,Gazélec Ajaccio,0,0.0,True,68.1,9.6
3193,36ee0b35-a98a-41fe-b820-d6f69b5b8f3b,3829437,"[41.1, 71.5]",1,26,Angers,NaN,Gazélec Ajaccio,0,0.0,True,41.1,71.5
3194,08addc99-bf82-4487-af28-5fc4d316732f,3829437,"[23.9, 56.7]",1,40,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,23.9,56.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
859377,093b8abf-d63f-44a8-99b1-953a4af3f184,3754031,"[29.4, 19.2]",2,176,Sunderland,NaN,Sunderland,1,0.0,NaN,29.4,19.2
859378,5bcdd974-0d85-46cc-8c00-d25aa5641d76,3754031,"[42.5, 33.5]",2,181,Manchester United,NaN,Sunderland,0,0.0,NaN,42.5,33.5
859379,dfd185df-5377-4631-a6bc-c207a1fcb376,3754031,"[62.0, 2.2]",2,187,Manchester United,NaN,Sunderland,0,0.0,NaN,62.0,2.2
859380,9a001277-eacb-466e-9eaf-562a1ae49116,3754031,"[93.3, 6.6]",2,189,Sunderland,NaN,Sunderland,1,0.0,NaN,93.3,6.6


In [ ]:
# Group by match_id
match_analysis_interception = df_final_matches_selected_interception.groupby('match_id').agg(
    interception_count=('id', 'count'),
    interception_proportion_period_1=('period', lambda x: (x == 1).mean()),
    interception_proportion_home_team=('home_team_flag', 'mean'),
    interception_proportion_under_pressure=('under_pressure', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
    interception_counterpress_rate=('counterpress', lambda x: x.fillna(False).mean()),
    interception_start_mean=('x', 'mean'),
    interception_start_proportion_x_lt_60=('x', lambda x: (x < 60).mean()), # Proportion of carry start x location when x is less than 60
    interception_start_y_between_20_60_proportion=('y', lambda y: ((y >= 20) & (y <= 60)).mean()) # Proportion of carry start y location when y is between 20 and 60
).reset_index()

match_analysis_interception

/tmp/ipython-input-46-3011905776.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  interception_proportion_under_pressure=('under_pressure', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
/tmp/ipython-input-46-3011905776.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  interception_counterpress_rate=('counterpress', lambda x: x.fillna(False).mean()),


,match_id,interception_count,interception_proportion_period_1,interception_proportion_home_team,interception_proportion_under_pressure,interception_counterpress_rate,interception_start_mean,interception_start_proportion_x_lt_60,interception_start_y_between_20_60_proportion
0,3753972,13,0.692308,0.538462,0.076923,0.153846,48.230769,0.692308,0.461538
1,3753973,33,0.545455,0.424242,0.212121,0.363636,43.363636,0.787879,0.454545
2,3753974,11,0.545455,0.727273,0.272727,0.363636,53.681818,0.545455,0.636364
3,3753975,53,0.415094,0.320755,0.169811,0.283019,40.713208,0.773585,0.603774
4,3753976,29,0.310345,0.482759,0.034483,0.206897,46.296552,0.655172,0.413793
...,...,...,...,...,...,...,...,...,...
245,3877072,20,0.400000,0.450000,0.050000,0.150000,56.185000,0.500000,0.500000
246,3877090,29,0.793103,0.413793,0.068966,0.344828,40.534483,0.758621,0.551724
247,3877115,26,0.461538,0.500000,0.038462,0.153846,49.015385,0.615385,0.615385
248,3877170,19,0.526316,0.684211,0.105263,0.315789,40.689474,0.789474,0.473684


**df_ball_recovery** - reviewed

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y'
]
# Check if the columns exist in the database
df_ball_recovery = new_dfs['df_ball_recovery']

existing_columns = [col for col in selected_columns if col in df_ball_recovery.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_ball_recovery = df_ball_recovery[existing_columns]
df_final_matches_selected_ball_recovery

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y
3052,42dfd8e5-1594-419d-8efe-5bbfc017cbeb,3829437,"[27.6, 7.0]",1,6,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,27.6,7.0
3053,296fe7e0-1531-424f-bf11-3fc836d5a216,3829437,"[49.0, 40.7]",1,8,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,49.0,40.7
3054,6077d517-ed11-4507-8c7f-dd1b90c53099,3829437,"[57.3, 69.3]",1,9,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,57.3,69.3
3055,2a06be39-939a-43ed-957a-144294615fa9,3829437,"[12.7, 41.9]",1,11,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,12.7,41.9
3056,77cf3f61-2207-4853-9e05-d834449b13e5,3829437,"[6.2, 24.1]",1,11,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,6.2,24.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
859474,d992eb94-21e5-41dd-aaab-28209faf78b5,3754031,"[7.0, 37.9]",2,185,Sunderland,NaN,Sunderland,1,0.0,NaN,7.0,37.9
859475,d8c97c31-be0f-47bc-86a4-8062668ccaed,3754031,"[63.1, 1.7]",2,188,Sunderland,NaN,Sunderland,1,0.0,NaN,63.1,1.7
859476,d427dc43-754b-4377-b4d5-3fa9e1edf587,3754031,"[23.6, 73.5]",2,190,Manchester United,True,Sunderland,0,0.0,NaN,23.6,73.5
859477,4e55bec2-caa5-48c1-ace2-a6577ded1ab4,3754031,"[92.3, 5.3]",2,191,Sunderland,NaN,Sunderland,1,0.0,NaN,92.3,5.3


In [ ]:
# Group by match_id
match_analysis_ball_recovery = df_final_matches_selected_ball_recovery.groupby('match_id').agg(
    ball_recovery_count=('id', 'count'),
    ball_recovery_proportion_period_1=('period', lambda x: (x == 1).mean()),
    ball_recovery_proportion_home_team=('home_team_flag', 'mean'),
    ball_recovery_proportion_under_pressure=('under_pressure', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
    dribble_start_mean=('x', 'mean'),
    dribble_start_proportion_x_lt_60=('x', lambda x: (x < 60).mean()), # Proportion of carry start x location when x is less than 60
    dribble_start_y_between_20_60_proportion=('y', lambda y: ((y >= 20) & (y <= 60)).mean()) # Proportion of carry start y location when y is between 20 and 60
).reset_index()

match_analysis_ball_recovery

/tmp/ipython-input-48-2136038655.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ball_recovery_proportion_under_pressure=('under_pressure', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure


,match_id,ball_recovery_count,ball_recovery_proportion_period_1,ball_recovery_proportion_home_team,ball_recovery_proportion_under_pressure,dribble_start_mean,dribble_start_proportion_x_lt_60,dribble_start_y_between_20_60_proportion
0,3753972,127,0.456693,0.401575,0.125984,49.504724,0.637795,0.566929
1,3753973,119,0.462185,0.478992,0.193277,57.791597,0.546218,0.579832
2,3753974,106,0.405660,0.471698,0.207547,48.681132,0.632075,0.650943
3,3753975,117,0.521368,0.461538,0.188034,51.411966,0.572650,0.632479
4,3753976,118,0.525424,0.618644,0.127119,44.632203,0.694915,0.677966
...,...,...,...,...,...,...,...,...
245,3877072,68,0.485294,0.632353,0.132353,43.548529,0.705882,0.764706
246,3877090,116,0.586207,0.465517,0.172414,57.171552,0.491379,0.577586
247,3877115,92,0.565217,0.565217,0.141304,59.168478,0.521739,0.673913
248,3877170,101,0.485149,0.554455,0.128713,62.659406,0.445545,0.712871


**df_dispossessed** - reviewed

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y'
]
# Check if the columns exist in the database
df_dispossessed = new_dfs['df_dispossessed']

existing_columns = [col for col in selected_columns if col in df_dispossessed.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_dispossessed = df_dispossessed[existing_columns]
df_final_matches_selected_dispossessed

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y
3264,8125770d-d75d-41b2-ad42-7a728d293bb9,3829437,"[89.8, 61.2]",1,18,Gazélec Ajaccio,True,Gazélec Ajaccio,1,0.0,NaN,89.8,61.2
3265,f69a39d0-f6a4-491f-9cbc-53c42e5aa1e0,3829437,"[85.2, 54.5]",1,38,Gazélec Ajaccio,True,Gazélec Ajaccio,1,0.0,NaN,85.2,54.5
3266,2f11ccb3-935f-45a1-bbab-91f7ec6e1a40,3829437,"[98.8, 63.3]",1,42,Gazélec Ajaccio,True,Gazélec Ajaccio,1,0.0,NaN,98.8,63.3
3267,3af77c0c-0ed5-433a-9c0f-c2687c888979,3829437,"[84.6, 14.7]",1,46,Gazélec Ajaccio,True,Gazélec Ajaccio,1,0.0,NaN,84.6,14.7
3268,32c67511-1a3c-44f9-adca-a1a63c38d820,3829437,"[111.7, 73.4]",1,56,Gazélec Ajaccio,True,Gazélec Ajaccio,1,0.0,NaN,111.7,73.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
859833,907d406b-89ee-42d0-9daf-822c5ff09f8e,3754031,"[106.1, 48.2]",2,111,Manchester United,True,Sunderland,0,0.0,NaN,106.1,48.2
859834,110ac92e-60b4-45c4-8159-93a814c2b26d,3754031,"[103.8, 48.1]",2,133,Sunderland,True,Sunderland,1,0.0,NaN,103.8,48.1
859835,bbdcd0aa-f3d9-4dea-814b-4bc695ec33ed,3754031,"[20.6, 2.7]",2,139,Sunderland,True,Sunderland,1,0.0,NaN,20.6,2.7
859836,aff1d042-ae14-4f8e-b1c5-c9139d9d47db,3754031,"[50.6, 6.9]",2,140,Sunderland,True,Sunderland,1,0.0,NaN,50.6,6.9


In [ ]:
# Group by match_id
match_analysis_dispossessed = df_final_matches_selected_dispossessed.groupby('match_id').agg(
    dispossessed_count=('id', 'count'),
    dispossessed_proportion_period_1=('period', lambda x: (x == 1).mean()),
    dispossessed_proportion_home_team=('home_team_flag', 'mean'),
    dispossessed_start_mean=('x', 'mean'),
    dispossessed_start_proportion_x_lt_60=('x', lambda x: (x < 60).mean()), # Proportion of carry start x location when x is less than 60
    dispossessed_start_y_between_20_60_proportion=('y', lambda y: ((y >= 20) & (y <= 60)).mean()) # Proportion of carry start y location when y is between 20 and 60
).reset_index()

match_analysis_dispossessed

,match_id,dispossessed_count,dispossessed_proportion_period_1,dispossessed_proportion_home_team,dispossessed_start_mean,dispossessed_start_proportion_x_lt_60,dispossessed_start_y_between_20_60_proportion
0,3753972,16,0.625000,0.500000,54.243750,0.687500,0.187500
1,3753973,29,0.655172,0.379310,79.396552,0.172414,0.482759
2,3753974,22,0.363636,0.500000,77.136364,0.272727,0.545455
3,3753975,23,0.652174,0.608696,70.891304,0.347826,0.347826
4,3753976,34,0.617647,0.470588,65.552941,0.352941,0.470588
...,...,...,...,...,...,...,...
245,3877072,16,0.312500,0.687500,85.731250,0.062500,0.687500
246,3877090,25,0.520000,0.320000,70.312000,0.400000,0.640000
247,3877115,25,0.640000,0.400000,66.484000,0.440000,0.560000
248,3877170,24,0.541667,0.541667,80.612500,0.291667,0.541667


**Foul Committed**

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'foul_committed_card'
]
# Check if the columns exist in the database
df_foul_committed = new_dfs['df_foul_committed']

existing_columns = [col for col in selected_columns if col in df_foul_committed.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_foul_committed = df_foul_committed[existing_columns]
df_final_matches_selected_foul_committed

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,foul_committed_card
2957,3f7a701e-9dc5-426a-8226-789a69b4f559,3829437,"[25.6, 68.1]",1,4,Angers,NaN,Gazélec Ajaccio,0,NaN
2958,b5f957d3-d736-49fb-9fef-3fad11951581,3829437,"[16.9, 75.1]",1,14,Angers,NaN,Gazélec Ajaccio,0,NaN
2959,deb4cd0d-c4fe-4ee2-992c-676be14a79d1,3829437,"[6.6, 9.4]",1,23,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,NaN
2960,cc02f677-a2a4-4cb4-ad88-ab4dcce328fc,3829437,"[26.2, 44.8]",1,26,Angers,NaN,Gazélec Ajaccio,0,Yellow Card
2961,b68fee3b-30c5-4ace-a03c-82c2ab828098,3829437,"[54.4, 14.1]",1,33,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,NaN
...,...,...,...,...,...,...,...,...,...,...
859927,f574698d-388c-4fe9-a863-8513fcd1769b,3754031,"[47.4, 6.7]",2,144,Sunderland,NaN,Sunderland,1,NaN
859928,bc82b46f-be79-4559-a48c-fd726aab20f2,3754031,"[64.3, 9.8]",2,175,Sunderland,NaN,Sunderland,1,NaN
859929,d3a15cd2-6d32-4528-8b19-082609653d10,3754031,"[77.7, 64.3]",2,180,Sunderland,NaN,Sunderland,1,NaN
859930,d83db437-ede0-4b63-9be0-d256080393b8,3754031,"[71.6, 70.6]",2,185,Sunderland,NaN,Sunderland,1,NaN


In [ ]:
# Group by match_id
match_analysis_foul_committed = df_final_matches_selected_foul_committed.groupby('match_id').agg(
    foul_committed_yellow_card_count=('foul_committed_card', lambda x: (x == 'Yellow Card').sum()),
    foul_committed_red_card_count=('foul_committed_card', lambda x: ((x == 'Red Card')| (x == 'Second Yellow')).sum()),
    foul_committed_cards_period_1_count=('foul_committed_card', lambda x: ((x[df_final_matches_selected_foul_committed.loc[x.index, 'period'] == 1] == 'Yellow Card') | (x[df_final_matches_selected_foul_committed.loc[x.index, 'period'] == 1] == 'Red Card') | (x[df_final_matches_selected_foul_committed.loc[x.index, 'period'] == 1] == 'Second Yellow')).sum())
).reset_index()

match_analysis_foul_committed

,match_id,foul_committed_yellow_card_count,foul_committed_red_card_count,foul_committed_cards_period_1_count
0,3753972,4,0,1
1,3753973,4,0,2
2,3753974,3,0,2
3,3753975,2,0,1
4,3753976,2,0,1
...,...,...,...,...
245,3877072,5,0,1
246,3877090,5,0,1
247,3877115,0,0,0
248,3877170,1,0,1


**df_foul_won** - on pause

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y',
    'foul_won_defensive',
    'foul_won_advantage'
]
# Check if the columns exist in the database
df_foul_won = new_dfs['df_foul_won']

existing_columns = [col for col in selected_columns if col in df_foul_won.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_foul_won = df_foul_won[existing_columns]
df_final_matches_selected_foul_won

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y,foul_won_defensive,foul_won_advantage
2982,b27b0039-b500-4807-b1b7-f73b7963b8d8,3829437,"[94.5, 12.0]",1,4,Angers,True,Gazélec Ajaccio,0,0.0,NaN,94.5,12.0,NaN,NaN
2983,3ae48903-effc-4de7-a2d4-52d75d1a46ef,3829437,"[103.2, 5.0]",1,14,Angers,True,Gazélec Ajaccio,0,0.0,NaN,103.2,5.0,NaN,NaN
2984,bf84a702-acf4-4716-b633-da6483828c45,3829437,"[113.5, 70.7]",1,23,Gazélec Ajaccio,True,Gazélec Ajaccio,1,0.0,NaN,113.5,70.7,NaN,NaN
2985,11996099-96ad-487e-ad35-06083fe31aa0,3829437,"[93.9, 35.3]",1,26,Angers,True,Gazélec Ajaccio,0,0.0,NaN,93.9,35.3,True,NaN
2986,eb316cf2-2059-4519-a0a8-8e44b3884bd0,3829437,"[65.7, 66.0]",1,33,Gazélec Ajaccio,True,Gazélec Ajaccio,1,0.0,NaN,65.7,66.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859950,1e225452-cc58-4b1b-98ea-b520669acfe3,3754031,"[72.7, 73.4]",2,144,Sunderland,True,Sunderland,1,0.0,NaN,72.7,73.4,NaN,NaN
859951,4e4c2ff3-061c-49ba-a774-662785ff9285,3754031,"[55.8, 70.3]",2,175,Sunderland,True,Sunderland,1,0.0,NaN,55.8,70.3,NaN,NaN
859952,c758d200-68d0-4b15-bc49-cbe990c49614,3754031,"[42.4, 15.8]",2,180,Sunderland,True,Sunderland,1,0.0,NaN,42.4,15.8,True,NaN
859953,2ab35937-5ad3-4964-bafb-652875dd893b,3754031,"[48.5, 9.5]",2,185,Sunderland,True,Sunderland,1,0.0,NaN,48.5,9.5,True,NaN


In [ ]:
# Group by match_id
match_analysis_foul_won = df_final_matches_selected_foul_won.groupby('match_id').agg(
    foul_won_count=('id', 'count'),
    foul_won_proportion_period_1=('period', lambda x: (x == 1).mean()),
    foul_won_proportion_home_team=('home_team_flag', 'mean'),
    foul_won_start_mean=('x', 'mean'),
    foul_won_start_proportion_x_lt_60=('x', lambda x: (x < 60).mean()), # Proportion of carry start x location when x is less than 60
    foul_won_start_y_between_20_60_proportion=('y', lambda y: ((y >= 20) & (y <= 60)).mean()), # Proportion of carry start y location when y is between 20 and 60
    foul_won_defensive_proportion=('foul_won_defensive', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
    foul_won_advantage_proportion=('foul_won_advantage', lambda x: x.fillna(False).mean()) # Handle potential NaNs in under_pressure
).reset_index()

match_analysis_foul_won

/tmp/ipython-input-54-3335432041.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  foul_won_defensive_proportion=('foul_won_defensive', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
/tmp/ipython-input-54-3335432041.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  foul_won_advantage_proportion=('foul_won_advantage', lambda x: x.fillna(False).mean()) # Handle potential NaNs in under_pressure


,match_id,foul_won_count,foul_won_proportion_period_1,foul_won_proportion_home_team,foul_won_start_mean,foul_won_start_proportion_x_lt_60,foul_won_start_y_between_20_60_proportion,foul_won_defensive_proportion,foul_won_advantage_proportion
0,3753972,21,0.428571,0.476190,57.704762,0.571429,0.333333,0.142857,0.142857
1,3753973,26,0.500000,0.538462,65.446154,0.500000,0.384615,0.307692,0.076923
2,3753974,34,0.529412,0.382353,56.073529,0.529412,0.647059,0.176471,0.176471
3,3753975,17,0.588235,0.470588,62.311765,0.529412,0.470588,0.294118,0.176471
4,3753976,20,0.450000,0.650000,67.670000,0.350000,0.400000,0.100000,0.100000
...,...,...,...,...,...,...,...,...,...
245,3877072,28,0.464286,0.464286,60.275000,0.500000,0.750000,0.250000,0.142857
246,3877090,31,0.548387,0.387097,50.551613,0.612903,0.516129,0.161290,0.064516
247,3877115,25,0.640000,0.160000,55.108000,0.520000,0.680000,0.200000,0.160000
248,3877170,27,0.370370,0.555556,56.729630,0.629630,0.703704,0.222222,0.148148


**df_error** - not to use

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y'
]
# Check if the columns exist in the database
df_error = new_dfs['df_error']

existing_columns = [col for col in selected_columns if col in df_error.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_error = df_error[existing_columns]
df_final_matches_selected_error

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y
10496,08a0e4e7-6563-4b15-923a-144f3660cf5d,3829435,"[10.0, 50.7]",1,29,Rennes,NaN,Lyon,0,0.0,NaN,10.0,50.7
13710,fc096197-a6c5-453b-916b-fb7a876e70cf,3829434,"[4.1, 42.8]",2,205,Bastia,NaN,Bastia,1,0.0,NaN,4.1,42.8
35523,ec4612c6-82b3-41da-8062-82687af013ac,3829426,"[20.8, 27.6]",2,163,Lyon,NaN,Guingamp,0,0.0,NaN,20.8,27.6
39232,9a09435d-9593-4d26-ac9d-d6714b937bea,3829425,"[36.8, 69.7]",1,34,Angers,NaN,Angers,1,0.0,NaN,36.8,69.7
56712,1ab2dcd5-4d2d-417a-bc11-ddad399cd81c,3829420,"[10.1, 40.4]",1,75,Bordeaux,NaN,Bordeaux,1,0.0,NaN,10.1,40.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
840265,cb5094d7-90bc-483e-9e1d-92e610a394e1,3754021,"[22.6, 56.3]",1,2,West Ham United,NaN,AFC Bournemouth,0,0.0,NaN,22.6,56.3
844005,a401a040-b6bf-4a9e-8daf-f208b096f041,3754022,"[5.2, 44.6]",1,80,Norwich City,NaN,Crystal Palace,0,0.0,NaN,5.2,44.6
847351,11f35ce8-3868-4307-850e-369d0193a150,3754024,"[11.0, 38.2]",2,158,Swansea City,NaN,Newcastle United,0,0.0,NaN,11.0,38.2
857037,242727d4-6a8a-4a03-94b3-e0a159b5f629,3754030,"[15.4, 56.0]",1,87,Watford,NaN,Watford,1,0.0,NaN,15.4,56.0


In [ ]:
# Group by match_id
match_analysis_error = df_final_matches_selected_error.groupby('match_id').agg(
    error_count=('id', 'count'),
    error_proportion_period_1=('period', lambda x: (x == 1).mean()),
    error_proportion_home_team=('home_team_flag', 'mean'),
    error_proportion_under_pressure=('under_pressure', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
    error_duration_mean=('duration', 'mean'),
    error_counterpress_rate=('counterpress', lambda x: x.fillna(False).mean()),
    dribble_start_mean=('x', 'mean'),
    dribble_start_proportion_x_lt_60=('x', lambda x: (x < 60).mean()), # Proportion of carry start x location when x is less than 60
    dribble_start_y_between_20_60_proportion=('y', lambda y: ((y >= 20) & (y <= 60)).mean()) # Proportion of carry start y location when y is between 20 and 60
).reset_index()

match_analysis_error

/tmp/ipython-input-56-2607876958.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  error_proportion_under_pressure=('under_pressure', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
/tmp/ipython-input-56-2607876958.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  error_counterpress_rate=('counterpress', lambda x: x.fillna(False).mean()),


,match_id,error_count,error_proportion_period_1,error_proportion_home_team,error_proportion_under_pressure,error_duration_mean,error_counterpress_rate,dribble_start_mean,dribble_start_proportion_x_lt_60,dribble_start_y_between_20_60_proportion
0,3753973,1,1.0,0.000000,0.0,0.0,0.0,26.90,1.0,1.0
1,3753977,2,0.5,0.000000,0.0,0.0,0.0,16.35,1.0,1.0
2,3753979,1,0.0,1.000000,0.0,0.0,0.0,26.40,1.0,1.0
3,3753983,1,0.0,0.000000,0.0,0.0,0.0,6.40,1.0,1.0
4,3753990,2,0.5,1.000000,0.0,0.0,0.0,9.40,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
79,3829434,1,0.0,1.000000,0.0,0.0,0.0,4.10,1.0,1.0
80,3829435,1,1.0,0.000000,0.0,0.0,0.0,10.00,1.0,1.0
81,3877090,1,0.0,0.000000,0.0,0.0,0.0,56.70,1.0,1.0
82,3877115,3,1.0,0.333333,0.0,0.0,0.0,18.50,1.0,1.0


**df_injury_stoppage** - reviewed

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y'
]
# Check if the columns exist in the database
df_injury_stoppage = new_dfs['df_injury_stoppage']

existing_columns = [col for col in selected_columns if col in df_injury_stoppage.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_injury_stoppage = df_injury_stoppage[existing_columns]
df_final_matches_selected_injury_stoppage

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y
3300,aa68a3d4-b0ab-4f95-8320-4dae457e65aa,3829437,NaN,1,66,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,NaN,NaN
3301,56547442-4255-45f2-8107-6f8d4ed4473e,3829437,NaN,1,66,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,NaN,NaN
3302,50e2826d-bc9b-404b-8e84-57282d5ca895,3829437,NaN,2,115,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,NaN,NaN
3303,d73a331c-ff79-46c5-902f-0fe6e2bb2b7c,3829437,NaN,2,159,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,NaN,NaN
3304,50d93358-7d60-4f4c-b46a-bf5bc873be22,3829437,NaN,2,176,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
860164,99fc44ea-cfd0-490c-860a-d78504f1468e,3754031,NaN,1,16,Manchester United,NaN,Sunderland,0,0.0,NaN,NaN,NaN
860165,10080ce6-a059-4c68-bbc8-637af53282bc,3754031,NaN,1,22,Manchester United,NaN,Sunderland,0,0.0,NaN,NaN,NaN
860166,1e9e58e7-757b-40e1-8f4f-74b993253a12,3754031,NaN,1,22,Manchester United,NaN,Sunderland,0,0.0,NaN,NaN,NaN
860167,39b56b38-d4f9-41a9-b9f7-ec22be1827ca,3754031,NaN,1,67,Manchester United,NaN,Sunderland,0,0.0,NaN,NaN,NaN


In [ ]:
# Group by match_id
match_analysis_injury_stoppage = df_final_matches_selected_injury_stoppage.groupby('match_id').agg(
    injury_stoppage_count=('id', 'count'),
    injury_stoppage_proportion_period_1=('period', lambda x: (x == 1).mean()),
    injury_stoppage_proportion_home_team=('home_team_flag', 'mean')
).reset_index()

match_analysis_injury_stoppage

,match_id,injury_stoppage_count,injury_stoppage_proportion_period_1,injury_stoppage_proportion_home_team
0,3753972,2,0.500000,0.500000
1,3753973,1,1.000000,1.000000
2,3753975,4,0.000000,0.000000
3,3753976,2,0.000000,1.000000
4,3753978,2,1.000000,1.000000
...,...,...,...,...
208,3877072,8,0.625000,0.625000
209,3877090,2,0.000000,0.000000
210,3877115,6,0.666667,0.666667
211,3877170,2,1.000000,0.000000


**df_referee_ball-drop** - reviewed

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y'
]
# Check if the columns exist in the database
df_drop_ball = new_dfs['df_referee_ball-drop']

existing_columns = [col for col in selected_columns if col in df_drop_ball.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_drop_ball = df_drop_ball[existing_columns]
df_final_matches_selected_drop_ball

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y
13686,3845124d-0cad-4e79-bcf7-0933e78c2b13,3829434,"[84.6, 25.3]",1,83,Guingamp,NaN,Bastia,0,0.0,NaN,84.6,25.3
13687,16f3b28a-5f13-4f81-9a3d-56e168b34f12,3829434,"[35.5, 54.8]",1,84,Bastia,NaN,Bastia,1,0.0,NaN,35.5,54.8
13688,3a2199e0-03a8-448b-b481-849f55db98c3,3829434,"[104.5, 9.2]",1,98,Bastia,NaN,Bastia,1,0.0,NaN,104.5,9.2
13689,c89f20d2-c274-48d8-9ee6-4bd749747e23,3829434,"[15.6, 70.9]",1,99,Guingamp,NaN,Bastia,0,0.0,NaN,15.6,70.9
17667,7029cda3-b352-4bca-bed4-aa7105f77668,3829432,"[70.1, 9.1]",1,38,Gazélec Ajaccio,NaN,Paris Saint-Germain,0,0.0,NaN,70.1,9.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
823030,b2d721a4-97a4-4bcc-905e-95db28740a3b,3754009,"[52.6, 44.9]",2,109,Arsenal,NaN,Arsenal,1,0.0,NaN,52.6,44.9
843995,7fed6714-2a0e-47d5-96c1-8caad3e1d89d,3754022,"[30.2, 14.6]",1,19,Crystal Palace,NaN,Crystal Palace,1,0.0,NaN,30.2,14.6
843996,21976948-a25f-4e4f-a4a6-adec49131cea,3754022,"[89.3, 65.5]",1,21,Norwich City,NaN,Crystal Palace,0,0.0,NaN,89.3,65.5
857053,654d5bc4-e97f-4a3a-8138-64210ae8fb04,3754030,"[48.2, 39.8]",2,116,Everton,NaN,Watford,0,0.0,NaN,48.2,39.8


In [ ]:
# Group by match_id and calculate total count, proportion by period, proportion by home_team_flag, and proportion by under_pressure
match_analysis_drop_ball = df_final_matches_selected_drop_ball.groupby('match_id').agg(
    drop_ball_count=('id', 'count')
).reset_index()

match_analysis_drop_ball

,match_id,drop_ball_count
0,3753972,2
1,3753975,1
2,3753976,2
3,3753984,2
4,3753988,4
...,...,...
81,3877072,4
82,3877090,2
83,3877115,4
84,3877170,2


**df_clearance** - reviewed

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y',
    'clearance_head',
    'clearance_left_foot',
    'clearance_right_foot'
]
# Check if the columns exist in the database
df_clearance = new_dfs['df_clearance']

existing_columns = [col for col in selected_columns if col in df_clearance.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_clearance = df_clearance[existing_columns]
df_final_matches_selected_clearance

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y,clearance_head,clearance_left_foot,clearance_right_foot
2794,3d5bdfe5-7386-4f15-864c-993ce754b470,3829437,"[10.2, 31.7]",1,3,Angers,True,Gazélec Ajaccio,0,0.0,NaN,10.2,31.7,NaN,NaN,True
2795,bf084085-d1f8-4986-a2e2-140fb4a15f79,3829437,"[10.0, 46.9]",1,3,Angers,True,Gazélec Ajaccio,0,0.0,NaN,10.0,46.9,NaN,NaN,True
2796,7eedd29f-aa90-435d-b1ea-5a654b0f215f,3829437,"[4.1, 32.8]",1,12,Gazélec Ajaccio,True,Gazélec Ajaccio,1,0.0,NaN,4.1,32.8,True,NaN,NaN
2797,b1791900-7c8f-4d46-b07e-40f9b515d92f,3829437,"[9.8, 40.8]",1,14,Angers,True,Gazélec Ajaccio,0,0.0,NaN,9.8,40.8,True,NaN,NaN
2798,c2d03b1c-1835-4d0d-8110-599ec92efdfb,3829437,"[9.4, 51.6]",1,14,Angers,True,Gazélec Ajaccio,0,0.0,NaN,9.4,51.6,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860066,f9aa9e68-0df1-4ea0-87c5-59464df5d0e5,3754031,"[38.0, 5.3]",2,183,Manchester United,True,Sunderland,0,0.0,NaN,38.0,5.3,NaN,True,NaN
860067,eb07e388-3b27-43af-93d3-2907e6234c4e,3754031,"[20.5, 72.2]",2,189,Sunderland,True,Sunderland,1,0.0,NaN,20.5,72.2,True,NaN,NaN
860068,77cc7ad9-c7f3-4580-b0cc-61b3d5bb70cd,3754031,"[55.8, 37.1]",2,190,Manchester United,True,Sunderland,0,0.0,NaN,55.8,37.1,True,NaN,NaN
860069,12b61fd5-fa8b-4b26-9bdc-20f3051ee4c5,3754031,"[43.2, 29.3]",2,192,Manchester United,True,Sunderland,0,0.0,NaN,43.2,29.3,True,NaN,NaN


In [ ]:
# Group by match_id
match_analysis_clearance = df_final_matches_selected_clearance.groupby('match_id').agg(
    clearance_count=('id', 'count'),
    clearance_proportion_period_1=('period', lambda x: (x == 1).mean()),
    clearance_proportion_home_team=('home_team_flag', 'mean'),
    clearance_start_mean=('x', 'mean'),
    clearance_start_proportion_x_lt_60=('x', lambda x: (x < 60).mean()), # Proportion of carry start x location when x is less than 60
    clearance_start_y_between_20_60_proportion=('y', lambda y: ((y >= 20) & (y <= 60)).mean()), # Proportion of carry start y location when y is between 20 and 60
    clearance_head_proportion=('clearance_head', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
    clearance_left_foot_proportion=('clearance_left_foot', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
    clearance_right_foot_proportion=('clearance_right_foot', lambda x: x.fillna(False).mean()) # Handle potential NaNs in under_pressure
).reset_index()

match_analysis_clearance

/tmp/ipython-input-62-357399348.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  clearance_head_proportion=('clearance_head', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
/tmp/ipython-input-62-357399348.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  clearance_left_foot_proportion=('clearance_left_foot', lambda x: x.fillna(False).mean()), # Handle potential NaNs in under_pressure
/tmp/ipython-input-62-357399348.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will c

,match_id,clearance_count,clearance_proportion_period_1,clearance_proportion_home_team,clearance_start_mean,clearance_start_proportion_x_lt_60,clearance_start_y_between_20_60_proportion,clearance_head_proportion,clearance_left_foot_proportion,clearance_right_foot_proportion
0,3753972,66,0.454545,0.469697,23.198485,0.954545,0.681818,0.348485,0.242424,0.393939
1,3753973,62,0.387097,0.612903,23.691935,0.967742,0.790323,0.645161,0.161290,0.177419
2,3753974,35,0.542857,0.228571,15.702857,1.000000,0.914286,0.371429,0.314286,0.314286
3,3753975,68,0.573529,0.529412,24.894118,1.000000,0.764706,0.573529,0.147059,0.264706
4,3753976,48,0.500000,0.708333,20.902083,0.937500,0.729167,0.500000,0.187500,0.291667
...,...,...,...,...,...,...,...,...,...,...
245,3877072,29,0.413793,0.724138,19.358621,1.000000,0.862069,0.517241,0.172414,0.310345
246,3877090,29,0.551724,0.758621,15.644828,0.931034,0.827586,0.379310,0.275862,0.344828
247,3877115,20,0.400000,0.450000,13.110000,1.000000,0.950000,0.400000,0.050000,0.550000
248,3877170,33,0.303030,0.545455,18.212121,0.969697,0.909091,0.454545,0.181818,0.363636


**df_offside** - reviewed

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y'
]
# Check if the columns exist in the database
df_offside = new_dfs['df_offside']

existing_columns = [col for col in selected_columns if col in df_offside.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_offside = df_offside[existing_columns]
df_final_matches_selected_offside

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y
7005,7f1467a7-260e-4707-93f0-3a3e901fde54,3829436,"[88.7, 4.6]",1,43,Stade de Reims,NaN,Nantes,0,0.0,NaN,88.7,4.6
21252,4aa597e0-d1c5-4dd4-9432-6f5305f52478,3829431,"[93.2, 24.8]",1,81,Lorient,NaN,Lorient,1,0.0,NaN,93.2,24.8
32419,9d362b0b-b0a9-4d2d-ab0c-177ec7b6aac9,3829427,"[105.9, 34.8]",2,94,OGC Nice,NaN,Troyes,0,0.0,NaN,105.9,34.8
39255,b6c505a6-8287-4f97-ab76-87774a4e907a,3829425,"[107.0, 51.3]",2,194,Angers,NaN,Angers,1,0.0,NaN,107.0,51.3
42391,fd81501d-1606-4f11-b317-9f049a999bb0,3829424,"[85.5, 30.5]",2,99,Saint-Étienne,NaN,Saint-Étienne,1,0.0,NaN,85.5,30.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
751343,def2630f-b10b-4bc3-94e1-4d5d09ce7efa,3754175,"[112.5, 46.5]",2,180,Sunderland,NaN,Sunderland,1,0.0,NaN,112.5,46.5
754821,7863e79d-ca38-45cc-9c92-8965addc7f7a,3753973,"[76.3, 65.5]",2,95,Chelsea,NaN,Chelsea,1,0.0,NaN,76.3,65.5
758575,1968b303-bb25-40af-a2c2-6316701c633b,3753975,"[117.8, 51.0]",1,70,Sunderland,NaN,Sunderland,1,0.0,NaN,117.8,51.0
791136,9dd912fd-6ae6-4d0d-8474-d5421bf5f334,3753990,"[112.9, 54.5]",2,147,Sunderland,NaN,Sunderland,1,0.0,NaN,112.9,54.5


In [ ]:
# Group by match_id
match_analysis_offside = df_final_matches_selected_offside.groupby('match_id').agg(
    offside_count=('id', 'count')
).reset_index()

match_analysis_offside

,match_id,offside_count
0,3753973,1
1,3753975,1
2,3753984,1
3,3753985,1
4,3753990,1
...,...,...
66,3829431,1
67,3829436,1
68,3877060,3
69,3877090,2


**df_dribbled_past** - reviewed

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y'
]
# Check if the columns exist in the database
df_dribbled_past = new_dfs['df_dribbled_past']

existing_columns = [col for col in selected_columns if col in df_dribbled_past.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_dribbled_past = df_dribbled_past[existing_columns]
df_final_matches_selected_dribbled_past

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y
3205,a810ff0c-d8ed-4587-9265-37061b58f01e,3829437,"[51.6, 13.9]",1,10,Angers,NaN,Gazélec Ajaccio,0,0.0,True,51.6,13.9
3206,9ff0d802-9e2c-496e-bf86-9f4bbd51f1cf,3829437,"[31.4, 75.1]",1,13,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,31.4,75.1
3207,7a1f9c07-3bc7-46db-a52f-290f0b95c9ec,3829437,"[44.7, 8.4]",1,47,Angers,NaN,Gazélec Ajaccio,0,0.0,True,44.7,8.4
3208,3c593be7-c074-4247-926d-8a94b812dfd8,3829437,"[81.4, 72.6]",1,72,Angers,NaN,Gazélec Ajaccio,0,0.0,True,81.4,72.6
3209,05229e3d-e0d1-4c2c-82d5-627a39183900,3829437,"[83.8, 75.8]",1,75,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,83.8,75.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
860087,d40cc49d-0ad2-4aff-97b6-7c55168ef2e5,3754031,"[61.0, 1.7]",2,171,Manchester United,NaN,Sunderland,0,0.0,NaN,61.0,1.7
860088,ec840d8a-05b0-4875-a987-d60cb512f1ef,3754031,"[78.4, 51.3]",2,182,Sunderland,NaN,Sunderland,1,0.0,True,78.4,51.3
860089,32bfc771-cb1e-43f2-8286-48a235b944c8,3754031,"[36.2, 77.7]",2,186,Sunderland,NaN,Sunderland,1,0.0,NaN,36.2,77.7
860090,912ba901-dad0-4ad5-b660-f4166c3cbdd7,3754031,"[60.2, 78.4]",2,188,Sunderland,NaN,Sunderland,1,0.0,True,60.2,78.4


In [ ]:
# Group by match_id
match_analysis_dribbled_past = df_final_matches_selected_dribbled_past.groupby('match_id').agg(
    dribbled_past_count=('id', 'count'),
    dribbled_past_proportion_period_1=('period', lambda x: (x == 1).mean()),
    dribbled_past_proportion_home_team=('home_team_flag', 'mean'),
    dribbled_past_counterpress_rate=('counterpress', lambda x: x.fillna(False).mean()),
    dribble_past_start_mean=('x', 'mean'),
    dribble_past_start_proportion_x_lt_60=('x', lambda x: (x < 60).mean()), # Proportion of carry start x location when x is less than 60
    dribble_past_start_y_between_20_60_proportion=('y', lambda y: ((y >= 20) & (y <= 60)).mean()) # Proportion of carry start y location when y is between 20 and 60
).reset_index()

match_analysis_dribbled_past

/tmp/ipython-input-66-3395002104.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dribbled_past_counterpress_rate=('counterpress', lambda x: x.fillna(False).mean()),


,match_id,dribbled_past_count,dribbled_past_proportion_period_1,dribbled_past_proportion_home_team,dribbled_past_counterpress_rate,dribble_past_start_mean,dribble_past_start_proportion_x_lt_60,dribble_past_start_y_between_20_60_proportion
0,3753972,18,0.444444,0.388889,0.277778,45.527778,0.666667,0.500000
1,3753973,31,0.677419,0.645161,0.161290,41.706452,0.677419,0.451613
2,3753974,40,0.350000,0.225000,0.225000,44.970000,0.700000,0.650000
3,3753975,18,0.388889,0.500000,0.277778,53.133333,0.722222,0.611111
4,3753976,29,0.448276,0.586207,0.275862,50.200000,0.620690,0.551724
...,...,...,...,...,...,...,...,...
245,3877072,18,0.388889,0.500000,0.333333,58.372222,0.444444,0.722222
246,3877090,28,0.571429,0.464286,0.285714,54.457143,0.571429,0.607143
247,3877115,19,0.421053,0.473684,0.210526,43.536842,0.736842,0.736842
248,3877170,25,0.320000,0.600000,0.160000,42.388000,0.760000,0.640000


**df_player_off** - reviewed

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y'
]
# Check if the columns exist in the database
df_player_off = new_dfs['df_player_off']

existing_columns = [col for col in selected_columns if col in df_player_off.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_off = df_player_off[existing_columns]
df_final_matches_selected_off

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y
3305,d4f8f1d9-921a-4688-8f42-d6b2b8a4f89a,3829437,NaN,1,66,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,NaN,NaN
3306,ab15d73e-1f74-4803-9f6b-11ce1a32aec8,3829437,NaN,2,115,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,NaN,NaN
3307,1181e335-3bd5-45b0-98b5-1a0e7a911e9f,3829437,NaN,2,159,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,NaN,NaN
3308,5f4cd631-f7d3-4266-b77f-978f872e63bc,3829437,NaN,2,176,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,NaN,NaN
17666,40a468bc-7f46-4c93-bb3c-d13f65bd8c00,3829432,NaN,1,37,Gazélec Ajaccio,NaN,Paris Saint-Germain,0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
808726,73eacc00-4461-491c-ba2f-67f789fb5a71,3753998,NaN,2,178,West Ham United,NaN,West Ham United,1,0.0,NaN,NaN,NaN
815630,cf614c32-832d-4dd6-be7b-5d5ba61c0d05,3754004,NaN,1,95,AFC Bournemouth,NaN,AFC Bournemouth,1,0.0,NaN,NaN,NaN
815631,f893f748-3131-42ca-8862-5fe428c8b2d4,3754004,NaN,2,134,AFC Bournemouth,NaN,AFC Bournemouth,1,0.0,NaN,NaN,NaN
834121,5c292ef2-5f08-4935-a5b0-ca2b4a5fdaed,3754018,NaN,2,116,Liverpool,NaN,Swansea City,0,0.0,NaN,NaN,NaN


In [ ]:
# Group by match_id
match_analysis_off = df_final_matches_selected_off.groupby('match_id').agg(
    off_count=('id', 'count')
).reset_index()

match_analysis_off

,match_id,off_count
0,3753972,1
1,3753973,1
2,3753977,1
3,3753985,1
4,3753993,1
...,...,...
86,3829431,2
87,3829432,1
88,3829437,4
89,3877072,2


**df_substitution** - reviewed

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y'
]
# Check if the columns exist in the database
df_substitution = new_dfs['df_substitution']

existing_columns = [col for col in selected_columns if col in df_substitution.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_substitution = df_substitution[existing_columns]
df_final_matches_selected_substitution

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y
3317,1fbfa56d-9352-4be2-b5bf-30a41312a948,3829437,NaN,2,98,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,NaN,NaN
3318,0915bb56-9850-4585-adab-8562deff4574,3829437,NaN,2,98,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,NaN,NaN
3319,4fabae47-43ae-4b91-b3c2-922f7c476cb5,3829437,NaN,2,139,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,NaN,NaN
3320,879a0601-ae31-4d40-bd2f-0d69ce22c803,3829437,NaN,2,144,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,NaN,NaN
3321,b0face55-1a42-4fcc-8213-9cae709a5c18,3829437,NaN,2,153,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
860199,c438f33c-2a60-4694-b632-f2099787a91b,3754031,NaN,1,67,Manchester United,NaN,Sunderland,0,0.0,NaN,NaN,NaN
860200,51c671b5-6fac-4ebe-a356-d4308195a7f6,3754031,NaN,2,123,Sunderland,NaN,Sunderland,1,0.0,NaN,NaN,NaN
860201,13b2d640-fe9b-4449-863d-e46aafc4c12b,3754031,NaN,2,144,Sunderland,NaN,Sunderland,1,0.0,NaN,NaN,NaN
860202,e600ee64-736c-4574-ac09-420c073d5b83,3754031,NaN,2,171,Manchester United,NaN,Sunderland,0,0.0,NaN,NaN,NaN


In [ ]:
# Group by match_id
match_analysis_substitution = df_final_matches_selected_substitution.groupby('match_id').agg(
    substitution_count=('id', 'count'),
    substitution_proportion_period_1=('period', lambda x: (x == 1).mean()),
    substitution_proportion_home_team=('home_team_flag', 'mean')
).reset_index()

match_analysis_substitution

,match_id,substitution_count,substitution_proportion_period_1,substitution_proportion_home_team
0,3753972,6,0.0,0.500000
1,3753973,6,0.0,0.666667
2,3753974,4,0.0,0.750000
3,3753975,5,0.0,0.200000
4,3753976,5,0.0,0.200000
...,...,...,...,...
245,3877072,9,0.0,0.333333
246,3877090,9,0.0,0.888889
247,3877115,10,0.4,0.300000
248,3877170,9,0.0,0.666667


**df_bad_behaviour** - reviewed

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y',
    'bad_behaviour_card'
]
# Check if the columns exist in the database
df_bad_behaviour = new_dfs['df_bad_behaviour']

existing_columns = [col for col in selected_columns if col in df_bad_behaviour.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_bad_behaviour = df_bad_behaviour[existing_columns]
df_final_matches_selected_bad_behaviour

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y,bad_behaviour_card
10511,238a4340-62d5-45a5-8ba8-dbe6c94a46b8,3829435,NaN,2,121,Lyon,NaN,Lyon,1,0.0,NaN,NaN,NaN,Yellow Card
21277,f69b75a9-746a-4898-837f-ef5b2f5d1c14,3829431,NaN,2,186,Bastia,NaN,Lorient,0,0.0,NaN,NaN,NaN,Yellow Card
28647,679c9304-1ba2-410b-9db0-85fd5ecd21a4,3829429,NaN,1,101,Rennes,NaN,Rennes,1,0.0,NaN,NaN,NaN,Yellow Card
28648,05a69ced-2ede-4c93-bebf-f3a4aef18d27,3829429,NaN,1,101,Rennes,NaN,Rennes,1,0.0,NaN,NaN,NaN,Yellow Card
32420,2256e538-c4ba-48a3-a2bf-d97dc58e3a24,3829427,NaN,2,167,Troyes,NaN,Troyes,1,0.0,NaN,NaN,NaN,Yellow Card
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776940,7b54e0ea-0307-4119-8c28-900d903df9e1,3753982,NaN,2,177,Southampton,NaN,Leicester City,0,0.0,NaN,NaN,NaN,Yellow Card
780449,9d5a02ec-b418-4765-b4cd-3d88c60afc47,3753986,NaN,2,166,Liverpool,NaN,Liverpool,1,0.0,NaN,NaN,NaN,Yellow Card
815640,d6f5e10a-d80f-4f19-b785-47e1e2549fcf,3754004,NaN,2,105,AFC Bournemouth,NaN,AFC Bournemouth,1,0.0,NaN,NaN,NaN,Yellow Card
841334,3fb8ba4f-c805-400c-8228-bc1eeda1a6ca,3754021,NaN,2,136,West Ham United,NaN,AFC Bournemouth,0,0.0,NaN,NaN,NaN,Yellow Card


In [ ]:
# Group by match_id
match_analysis_bad_behaviour = df_final_matches_selected_bad_behaviour.groupby('match_id').agg(
    bad_behaviour_count=('id', 'count'),
    bad_behaviour_proportion_period_1=('period', lambda x: (x == 1).mean()),
    bad_behaviour_proportion_home_team=('home_team_flag', 'mean'),
    bad_behaviour_yellow_card_proportion=('bad_behaviour_card', lambda x: (x == 'Yellow Card').mean())
).reset_index()

match_analysis_bad_behaviour

,match_id,bad_behaviour_count,bad_behaviour_proportion_period_1,bad_behaviour_proportion_home_team,bad_behaviour_yellow_card_proportion
0,3753972,1,0.00,0.0,1.0
1,3753973,4,0.25,0.5,1.0
2,3753974,1,0.00,1.0,1.0
3,3753979,1,0.00,1.0,1.0
4,3753982,1,0.00,0.0,1.0
...,...,...,...,...,...
86,3877060,1,0.00,0.0,1.0
87,3877072,1,1.00,0.0,1.0
88,3877090,2,0.00,0.5,1.0
89,3877170,1,0.00,1.0,1.0


**df_tactical_shift** - reviewed

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'duration',
    'counterpress',
    'x',
    'y'
]
# Check if the columns exist in the database
df_tactical_shift = new_dfs['df_tactical_shift']

existing_columns = [col for col in selected_columns if col in df_tactical_shift.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_tactical_shift = df_tactical_shift[existing_columns]
df_final_matches_selected_tactical_shift

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,duration,counterpress,x,y
3323,3a6b114e-a64b-4df6-8c86-0ed5461b7601,3829437,NaN,2,99,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,NaN,NaN
3324,0d8c7e72-3824-4e42-8d0c-ab882c82336b,3829437,NaN,2,99,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,NaN,NaN
3325,e879cac9-f3b2-4f8c-b1c3-5a605d24ec66,3829437,NaN,2,146,Gazélec Ajaccio,NaN,Gazélec Ajaccio,1,0.0,NaN,NaN,NaN
3326,12ae8fac-7c00-47f6-b907-c10498bbdb12,3829437,NaN,2,188,Angers,NaN,Gazélec Ajaccio,0,0.0,NaN,NaN,NaN
7006,f3727d5e-9e29-4c80-9a8a-2dd7c03e9fba,3829436,NaN,1,65,Nantes,NaN,Nantes,1,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
850163,2cf3e809-5789-4ce8-9d94-38987a7d0f60,3754026,NaN,2,135,Crystal Palace,NaN,Crystal Palace,1,0.0,NaN,NaN,NaN
850164,9b9b4d78-4336-45f3-aaeb-1af5518dc298,3754026,NaN,2,182,Crystal Palace,NaN,Crystal Palace,1,0.0,NaN,NaN,NaN
853663,c7cbe3f5-3acd-4bf0-a463-4f78999eed55,3754027,NaN,2,131,Sunderland,NaN,Tottenham Hotspur,0,0.0,NaN,NaN,NaN
860211,4d32e68e-7c85-4fe1-b94d-87a5f13477e1,3754031,NaN,2,123,Sunderland,NaN,Sunderland,1,0.0,NaN,NaN,NaN


In [ ]:
# Group by match_id
match_analysis_tactical_shift = df_final_matches_selected_bad_behaviour.groupby('match_id').agg(
    tactical_shift_count=('id', 'count'),
    tactical_shift_proportion_period_1=('period', lambda x: (x == 1).mean()),
    tactical_shift_proportion_home_team=('home_team_flag', 'mean')
).reset_index()

match_analysis_tactical_shift

,match_id,tactical_shift_count,tactical_shift_proportion_period_1,tactical_shift_proportion_home_team
0,3753972,1,0.00,0.0
1,3753973,4,0.25,0.5
2,3753974,1,0.00,1.0
3,3753979,1,0.00,1.0
4,3753982,1,0.00,0.0
...,...,...,...,...
86,3877060,1,0.00,0.0
87,3877072,1,1.00,0.0
88,3877090,2,0.00,0.5
89,3877170,1,0.00,1.0


In [ ]:

# List of dataframes to join
analysis_dfs = [
    match_analysis,
    df_wwc_subset,
    match_analysis_pass,
    match_analysis_carry,
    match_analysis_pressure,
    match_analysis_miscontrol,
    match_analysis_dribble,
    match_analysis_duel,
    match_analysis_interception,
    match_analysis_ball_recovery,
    match_analysis_dispossessed,
    match_analysis_foul_committed,
    match_analysis_foul_won,
    match_analysis_injury_stoppage,
    match_analysis_drop_ball,
    match_analysis_clearance,
    match_analysis_offside,
    match_analysis_dribbled_past,
    match_analysis_off,
    match_analysis_substitution,
    match_analysis_bad_behaviour,
    match_analysis_tactical_shift,
    match_analysis_shot
]

# Start with the first dataframe
match_analysis = analysis_dfs[0]

# Join the remaining dataframes iteratively
for df in analysis_dfs[1:]:
    match_analysis = pd.merge(match_analysis, df, on='match_id', how='outer')

# Display the merged dataframe
print(match_analysis.head())

   match_id  duration  xg  home_team_duration  max_posession  match_week  \
0      9575       NaN NaN                 NaN            NaN           1   
1      9581       NaN NaN                 NaN            NaN           2   
2      9592       NaN NaN                 NaN            NaN           3   
3      9602       NaN NaN                 NaN            NaN           4   
4      9609       NaN NaN                 NaN            NaN           5   

   kick_off_hour  pass_count  pass_proportion_period_1  \
0           20.0         NaN                       NaN   
1           18.0         NaN                       NaN   
2           20.0         NaN                       NaN   
3           16.0         NaN                       NaN   
4           22.0         NaN                       NaN   

   pass_proportion_home_team  ...  shot_play_pattern_proportion_regular_play  \
0                        NaN  ...                                        NaN   
1                        NaN  ... 

In [ ]:

# Specify the path where you want to save the CSV file in your Google Drive

file_path = f'/content/drive/My Drive/analysis_dfsv2{b}.csv'
file_path2 = f'/content/drive/My Drive/shot_analysis_dfsv2{b}.csv'
file_path3 = f'/content/drive/My Drive/possesion_start_dfsv2{b}.csv'
# Save the DataFrame to a CSV file
match_analysis.to_csv(file_path, index=False)
df_final_matches_selected_shot.to_csv(file_path2, index=False)
df_first_event_of_possession.to_csv(file_path3, index=False)
print(f"DataFrame saved successfully to {file_path}")
print(f"DataFrame saved successfully to {file_path2}")
print(f"DataFrame saved successfully to {file_path3}")

Mounted at /content/drive
DataFrame saved successfully to /content/drive/My Drive/analysis_dfsv28.csv
DataFrame saved successfully to /content/drive/My Drive/shot_analysis_dfsv28.csv
DataFrame saved successfully to /content/drive/My Drive/possesion_start_dfsv28.csv
